In [ ]:
!pip install utm
!pip install geojson 
!pip install geopandas
!pip install haversine
!pip install folium #지도 시각화 

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import utm
from haversine import haversine
from shapely.geometry import Point, Polygon, LineString
import math
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
Bike_Station=pd.read_csv('/content/drive/My Drive/Compas/compas/SBJ_2007_001/02.자전거스테이션.csv')
space_plan=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/10.도시계획(공간시설).geojson')
culphy_plan=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/11.도시계획(공공문화제육시설).geojson')
traffic_plan=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/12.도시계획(교통시설).geojson')
wetland_cons=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/13.용도지역지구(습지보호지역).geojson')
build_road=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/15.도로명주소_건물.geojson')
road_road=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/16.도로명주소_도로.geojson')
build_dist=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/17.일반건물 분포도(100M X 100M).geojson')
fes_space=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/18.행사장_공간정보.csv')
subway_station=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/19.전철역_공간정보.csv')
bus_station=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/20.고양시 버스정류소.csv')
parking_info=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/22.주차장정보.csv')
theat_muse=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/26.고양시 공연장 박물관 정보.csv')
physi_curr=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/27.고양시 체육시설 현황 정보.csv')
duk_godo=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/30.고양시_덕양구_고도.geojson')
ilsansu_godo=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/32.고양시_일산서구_고도.geojson')
ilsando_godo=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/31.고양시_일산동구_고도.geojson')
goyang_road=gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/33.고양시_인도.geojson')
gu_boundary = gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/08.행정경계(시군구).geojson')
address_code = pd.read_excel('/content/drive/My Drive/Compas/compas/SBJ_2007_001/28.코드정의서.xlsx')
df_pop = gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/06.인구(거주)분포도(100M X 100M).geojson')

In [ ]:
df_bus = pd.read_csv('/content/drive/My Drive/Compas/compas/csv 파일 여기로!/bus_merged.csv')
df_sub = pd.read_csv('/content/drive/My Drive/Compas/compas/csv 파일 여기로!/subway_merged.csv')
bike_rental_month = pd.read_csv('/content/drive/My Drive/Compas/compas/준수씨 자전거 자료/Koyang_Bike(정리중)_CSV및 다양한 파일/월별 대여 스테이션 이용 현황(완성).csv')
bike_return_month = pd.read_csv('/content/drive/My Drive/Compas/compas/준수씨 자전거 자료/Koyang_Bike(정리중)_CSV및 다양한 파일/월별 반납 스테이션 이용 현황(완성).csv')
all_plan = pd.read_csv("/content/drive/My Drive/Compas/compas/csv 파일 여기로!/location_all_final.csv")
real_plan = pd.read_csv('/content/drive/My Drive/Compas/compas/csv 파일 여기로!/location_real_final.csv')

In [ ]:
# lscp raw데이터들을 저장할 폴더가 없다면 만들기 
import os
try:
  os.mkdir('./LSCP')
except:
  pass

In [ ]:
# lscp 최종 모델용 파일을 저장할 폴더가 없다면 만들기 
import os
try:
  os.mkdir('./모델학습용 데이터')
except:
  pass

# 1. 고양시 전체 100m 단위 좌표 구하기 

In [ ]:
gu_boundary = gpd.read_file('/content/drive/My Drive/Compas/compas/SBJ_2007_001/08.행정경계(시군구).geojson')
gu_boundary.head()

,SIG_CD,SIG_KOR_NM,geometry
0,41281,고양시덕양구,"MULTIPOLYGON (((126.92910 37.74849, 126.92964 ..."
1,41285,고양시일산동구,"MULTIPOLYGON (((126.77762 37.67899, 126.77825 ..."
2,41287,고양시일산서구,"MULTIPOLYGON (((126.76703 37.70717, 126.76719 ..."


In [ ]:
from shapely.ops import unary_union
from tqdm.notebook import tqdm
# 구 경계 합쳐서 고양시 경계 구하기 
city_boundary = unary_union(gu_boundary['geometry']).bounds
y_min, x_min, y_max, x_max = city_boundary

# 기준 경계 
geometry = unary_union(gu_boundary['geometry'])

# 기준 경계 이내 100m 단위로 좌표 찍기 
def get_pointlist(y_min, x_min, y_max, x_max, geometry):
  y1, x1, _, _ = utm.from_latlon(x_min, y_min)
  y2, x2, _, _ = utm.from_latlon(x_min, y_max)
  y3, x3, _, _ = utm.from_latlon(x_max, y_min)
  y4, x4, _, _ = utm.from_latlon(x_max, y_max)

  x_min_utm = min(x1, x2, x3, x4) 
  x_max_utm = max(x1, x2, x3, x4)
  y_min_utm = min(y1, y2, y3, y4)
  y_max_utm = max(y1, y2, y3, y4)

  x_utm_points = np.arange(x_min_utm, x_max_utm+50, 100)
  y_utm_points = np.arange(y_min_utm, y_max_utm+50, 100)

  point_list = []
  for x in tqdm(x_utm_points):
    for y in y_utm_points: 
      point = utm.to_latlon(y, x, 52, 'S')
      if geometry.contains(Point(point[1], point[0])):
        point_list.append(point)
    
  return point_list

In [ ]:
point_list = get_pointlist(y_min, x_min, y_max, x_max, geometry)
len(point_list) # 총 26673개의 좌표를 구함 

26673

In [ ]:
# point들 위치 확인 (무작위로 3000개만 출력)
random_indices = np.random.choice(len(point_list), size = 3000, replace=False)
check_loc = np.array(point_list)[random_indices, :]

gu_boundary['색상'] = 1

# 100m 기준 격자 
import folium
m = folium.Map(location=(37.66714305514331, 126.82008045013437), zoom_start=12)
folium.Choropleth(
    geo_data = gu_boundary,
    data = gu_boundary,
    columns = ('SIG_KOR_NM', '색상'),
    key_on = 'feature.properties.SIG_KOR_NM',
    fill_color = 'BuPu',
    legend_name = '색상'
).add_to(m) #구 경계 표시 

for sub_lat, sub_long in check_loc: 
  folium.CircleMarker([sub_lat, sub_long],
       radius = 0.00000002, color = 'purple' ).add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
 m.save('/content/drive/My Drive/Compas/TEST/고양시 전체 좌표.html')

# 2. 좌표 당 거주인구 계산

In [ ]:
# 좌표 당 거주인구수 계산 함수 

## 인구 격자와의 최소 위치가 500m 이면 인구에 포함 ##
from tqdm.notebook import tqdm

def point_resident(location, df_pop):
  lscp=[]
  target_nm=df_pop.columns.tolist()[[(t.lower().find('gid')>=0) for t in df_pop.columns.tolist()].index(True)]
  for lat, long in tqdm(location):
    for nm, geometry in zip(df_pop[target_nm],df_pop['geometry']):
        h=[]
        x = (lat, long)
        points = [point for polygon in geometry for point in polygon.exterior.coords[:]]
        for t_long, t_lat in points:
          y=(t_lat, t_long)
          h.append(haversine(x,y, unit='m'))
        if min(h)<=300:
          lscp.append([x ,nm,min(h)])
  return lscp

In [ ]:
lscp_res = point_resident(point_list, df_pop)
geo_pop = pd.DataFrame(lscp_res, columns=['location', 'gid', 'haversine'])
df_resident=geo_pop.merge(df_pop, on='gid').groupby('location')[['location', 'val']].sum().reset_index()
df_resident.columns = ['좌표', '주거인구']

In [ ]:
df_resident

,좌표,주거인구
0,"(37.572900940384436, 126.85261001624662)",0.0
1,"(37.57378106068045, 126.85145247482409)",0.0
2,"(37.573801661159955, 126.85258414110766)",0.0
3,"(37.57382225080074, 126.85371580854316)",0.0
4,"(37.57466116865266, 126.85029490618022)",0.0
5,"(37.57468178063709, 126.85142658492211)",0.0
6,"(37.574702381782494, 126.85255826481664)",0.0
7,"(37.57472297208882, 126.85368994586322)",0.0
8,"(37.574743551556054, 126.85482162806123)",0.0
9,"(37.57476412018415, 126.85595331141019)",0.0


In [ ]:
df_resident['좌표'] = df_resident['좌표'].astype(str) # merge를 위해 데이터 타입 통일 
df_resident = lscp_base.merge(df_resident, how = 'left').fillna(0) #결측치는 0으로 
df_resident

,좌표,주거인구
0,"(37.57961579548528, 126.87506406643347)",0.0
1,"(37.57963617229605, 126.87619583846381)",0.0
2,"(37.57965653826528, 126.87732761163377)",0.0
3,"(37.58049614012486, 126.87390667102693)",0.0
4,"(37.5805165284361, 126.8750384555327)",0.0
...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0.0
20833,"(37.747040166676285, 126.91454518217792)",0.0
20834,"(37.74706024022562, 126.9156795278339)",0.0
20835,"(37.60951982027619, 126.83456514393055)",0.0


In [ ]:
df_resident.to_csv('./LSCP/전체좌표_resident.csv')

#LSCP_BASE : 데이터 프레임 merge틀

In [ ]:
# 앞으로 만들 lscp를 merge하기 위한 틀 만들기 
lscp_base = df_resident[['좌표']].copy()
lscp_base = lscp_base.astype(str)
lscp_base

,좌표
0,"(37.57961579548528, 126.87506406643347)"
1,"(37.57963617229605, 126.87619583846381)"
2,"(37.57965653826528, 126.87732761163377)"
3,"(37.58049614012486, 126.87390667102693)"
4,"(37.5805165284361, 126.8750384555327)"
...,...
20832,"(37.74702008221923, 126.91341083764837)"
20833,"(37.747040166676285, 126.91454518217792)"
20834,"(37.74706024022562, 126.9156795278339)"
20835,"(37.60951982027619, 126.83456514393055)"


# LSCP_ALL: LSCP 계산을 위한 틀 

In [ ]:
lscp_all = lscp_base.copy()
mapper = lambda x: float(str(x).strip('\(').strip('\)').split(',')[0])
lscp_all['위도'] = lscp_all['좌표'].map(mapper)

mapper = lambda x: float(str(x).strip('\(').strip('\)').split(',')[1])
lscp_all['경도'] = lscp_all['좌표'].map(mapper)

lscp_all

,좌표,위도,경도
0,"(37.57961579548528, 126.87506406643347)",37.579616,126.875064
1,"(37.57963617229605, 126.87619583846381)",37.579636,126.876196
2,"(37.57965653826528, 126.87732761163377)",37.579657,126.877328
3,"(37.58049614012486, 126.87390667102693)",37.580496,126.873907
4,"(37.5805165284361, 126.8750384555327)",37.580517,126.875038
...,...,...,...
20832,"(37.74702008221923, 126.91341083764837)",37.747020,126.913411
20833,"(37.747040166676285, 126.91454518217792)",37.747040,126.914545
20834,"(37.74706024022562, 126.9156795278339)",37.747060,126.915680
20835,"(37.60951982027619, 126.83456514393055)",37.609520,126.834565


#2. 좌표 별 유동인구 계산 (버스, 지하철)

In [ ]:
# 좌표 별 유동인구 계산 함수 
def point_foottraffic(location, df_bus, df_sub):
  sub_lscp = []
  bus_lscp = []
  # 지하철 2018년 기준 승하차수 
  for x in tqdm(location): 
    for i in range(len(df_sub)):
      y = (df_sub['지하철_위도'].iloc[i], df_sub['지하철_경도'].iloc[i])
      h = haversine(x,y, unit='m')
      if h <= 300:
        sub_lscp.append([x, df_sub['역명'].iloc[i], h])
    sub_pop = pd.DataFrame(sub_lscp, columns = ['location', '역명', 'haversine'])
    sub_merge = sub_pop.merge(df_sub, on = '역명').groupby('location')[['location', '승하차_연간합계']].sum().reset_index()
    sub_merge.columns = ['좌표', '지하철_승하차수']

  # 버스 최근 1년치 정류장별 승하차 
  for x in tqdm(location):
    for i in range(len(df_bus)):
      y = (df_bus['버스_위도'].iloc[i], df_bus['버스_경도'].iloc[i])
      h = haversine(x,y, unit='m')
      if h <= 300:
        bus_lscp.append([x, df_bus['정류장_ID'].iloc[i], h])
    bus_pop = pd.DataFrame(bus_lscp, columns = ['location', '정류장_ID', 'haversine'])
    bus_merge = bus_pop.merge(df_bus, on = '정류장_ID').groupby('location')[['location', '승차수']].sum().reset_index()
    bus_merge.columns = ['좌표', '버스_승하차수']


  merge = bus_merge.merge(sub_merge, how ='outer', on = '좌표').fillna(0)
  merge['유동인구'] = merge['버스_승하차수'] + merge['지하철_승하차수']
  
  return merge # 버스, 지하철, 유동인구 원하는걸로 출력 가능 

In [ ]:
df_foot = point_foottraffic(point_list, df_bus, df_sub)
df_foot

,버스_승하차수,좌표,지하철_승하차수,유동인구


In [ ]:
df_foot['좌표'] = df_foot['좌표'].astype(str) #데이터 프레임 merge를 위해 type 맞춰주기 

In [ ]:
df_foot = lscp_base.merge(df_foot, on = '좌표', how = 'left').fillna(0) #유동인구 데이터 좌표별로 merge 
df_foot

,좌표,버스_승하차수,지하철_승하차수,유동인구
0,"(37.57961579548528, 126.87506406643347)",0.0,0.0,0.0
1,"(37.57963617229605, 126.87619583846381)",0.0,0.0,0.0
2,"(37.57965653826528, 126.87732761163377)",0.0,0.0,0.0
3,"(37.58049614012486, 126.87390667102693)",0.0,0.0,0.0
4,"(37.5805165284361, 126.8750384555327)",0.0,0.0,0.0
...,...,...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0.0,0.0,0.0
20833,"(37.747040166676285, 126.91454518217792)",0.0,0.0,0.0
20834,"(37.74706024022562, 126.9156795278339)",0.0,0.0,0.0
20835,"(37.60951982027619, 126.83456514393055)",0.0,0.0,0.0


In [ ]:
# 데이터 저장 
df_foot.to_csv('./LSCP/전체좌표_foottraffic300.csv')

# 3. 고도 데이터


In [ ]:
def geo_godo(lscp_pop, target):
  lscp=[]
  for sta, lat, long in tqdm(zip(lscp_pop['좌표'],lscp_pop['위도'],lscp_pop['경도'])):
    for dn, geometry in zip(target['DN'],target['geometry']):
      h=[]
      x = (lat, long)
      points = [point for point in geometry.exterior.coords[:]] #해당 geometry의 테두리의 모든 점을 리스트로 받음 
      for t_long, t_lat in points:
        y=(t_lat, t_long)
        h.append(haversine(x,y, unit='m'))
      if min(h)<=300:
        lscp.append([sta,dn,min(h)]) #300m 이내에 있는 점 추출 
  return lscp

In [ ]:
duk_godo

,fid,DN,geometry
0,1,347,"POLYGON ((126.92421 37.75297, 126.92421 37.752..."
1,2,361,"POLYGON ((126.92523 37.75297, 126.92523 37.752..."
2,3,374,"POLYGON ((126.92625 37.75297, 126.92625 37.751..."
3,4,425,"POLYGON ((126.92727 37.75297, 126.92727 37.752..."
4,5,484,"POLYGON ((126.92829 37.75297, 126.92829 37.752..."
...,...,...,...
19103,19104,12,"POLYGON ((126.85308 37.56883, 126.85308 37.568..."
19104,19105,11,"POLYGON ((126.85410 37.56883, 126.85410 37.567..."
19105,19106,5,"POLYGON ((126.85206 37.56802, 126.85207 37.567..."
19106,19107,9,"POLYGON ((126.85308 37.56802, 126.85308 37.567..."


In [ ]:
ilsando_godo

,fid,DN,geometry
0,1,12,"POLYGON ((126.79284 37.73975, 126.79284 37.738..."
1,2,11,"POLYGON ((126.79488 37.73975, 126.79488 37.738..."
2,3,17,"POLYGON ((126.79080 37.73894, 126.79080 37.738..."
3,4,15,"POLYGON ((126.79182 37.73894, 126.79182 37.738..."
4,5,18,"POLYGON ((126.79284 37.73894, 126.79284 37.738..."
...,...,...,...
5693,5694,3,"POLYGON ((126.79316 37.62298, 126.79317 37.622..."
5694,5695,5,"POLYGON ((126.79418 37.62299, 126.79419 37.622..."
5695,5696,10,"POLYGON ((126.78909 37.62217, 126.78909 37.621..."
5696,5697,9,"POLYGON ((126.79011 37.62217, 126.79011 37.621..."


In [ ]:
ilsansu_godo

,fid,DN,geometry
0,1,19,"POLYGON ((126.76181 37.71165, 126.76182 37.710..."
1,2,20,"POLYGON ((126.76283 37.71165, 126.76284 37.709..."
2,3,22,"POLYGON ((126.76385 37.71165, 126.76386 37.710..."
3,4,25,"POLYGON ((126.76487 37.71165, 126.76488 37.710..."
4,5,27,"POLYGON ((126.76589 37.71165, 126.76590 37.710..."
...,...,...,...
2462,2463,2,"POLYGON ((126.73041 37.64590, 126.73041 37.644..."
2463,2464,3,"POLYGON ((126.73143 37.64590, 126.73143 37.644..."
2464,2465,5,"POLYGON ((126.73245 37.64590, 126.73245 37.644..."
2465,2466,2,"POLYGON ((126.73143 37.64428, 126.73144 37.643..."


In [ ]:
bike_duk=pd.DataFrame(geo_godo(lscp_all,duk_godo),columns=['좌표','DN','Harv_dis'])
bike_ildo=pd.DataFrame(geo_godo(lscp_all,ilsando_godo),columns=['좌표','DN','Harv_dis'])
bike_ilsu=pd.DataFrame(geo_godo(lscp_all,ilsansu_godo),columns=['좌표','DN','Harv_dis'])

In [ ]:
#lscp_base와 병합 이후 차례로 병합
bike_ilsu['좌표'] = bike_ilsu['좌표'].astype(str)
bike_duk['좌표'] = bike_duk['좌표'].astype(str)
bike_ildo['좌표'] = bike_ildo['좌표'].astype(str)

base_duk=lscp_base.merge(bike_duk.iloc[:,:-1],on='좌표',how='outer')
duk_ildo=base_duk.merge(bike_ildo.iloc[:,:-1],on='좌표',how='outer')
ildo_ilsu=duk_ildo.merge(bike_ilsu.iloc[:,:-1],on='좌표',how='outer')

In [ ]:
#nan값 채워주기
ildo_ilsu=ildo_ilsu.fillna(0)

In [ ]:
#평균을 구하기 위한 물밑작업
ildo_ilsu['DN_avg']=0.0

In [ ]:
ildo_ilsu

,좌표,DN_x,DN_y,DN,DN_avg
0,"(37.57961579548528, 126.87506406643347)",9.0,0,0,0.0
1,"(37.57961579548528, 126.87506406643347)",11.0,0,0,0.0
2,"(37.57961579548528, 126.87506406643347)",10.0,0,0,0.0
3,"(37.57961579548528, 126.87506406643347)",16.0,0,0,0.0
4,"(37.57961579548528, 126.87506406643347)",11.0,0,0,0.0
...,...,...,...,...,...
21468,"(37.74702008221923, 126.91341083764837)",0.0,0,0,0.0
21469,"(37.747040166676285, 126.91454518217792)",0.0,0,0,0.0
21470,"(37.74706024022562, 126.9156795278339)",0.0,0,0,0.0
21471,"(37.60951982027619, 126.83456514393055)",0.0,0,0,0.0


In [ ]:
# 값이 존재할 경우에만 count를 더해 실질적인 평균값이 나오도록 설계
for idx,i in enumerate(ildo_ilsu['DN_avg']):
  count=0
  if ildo_ilsu['DN'][idx]!=0:
    count+=1
  if ildo_ilsu['DN_x'][idx]!=0:
    count+=1
  if ildo_ilsu['DN_y'][idx]!=0:
    count+=1
  ildo_ilsu['DN_avg'][idx]=(ildo_ilsu['DN'][idx]+ildo_ilsu['DN_x'][idx]+ildo_ilsu['DN_y'][idx])/count

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
ildo_ilsu

,좌표,DN_x,DN_y,DN,DN_avg
0,"(37.57961579548528, 126.87506406643347)",9.0,0,0,9.0
1,"(37.57961579548528, 126.87506406643347)",11.0,0,0,11.0
2,"(37.57961579548528, 126.87506406643347)",10.0,0,0,10.0
3,"(37.57961579548528, 126.87506406643347)",16.0,0,0,16.0
4,"(37.57961579548528, 126.87506406643347)",11.0,0,0,11.0
...,...,...,...,...,...
21468,"(37.74702008221923, 126.91341083764837)",0.0,0,0,NaN
21469,"(37.747040166676285, 126.91454518217792)",0.0,0,0,NaN
21470,"(37.74706024022562, 126.9156795278339)",0.0,0,0,NaN
21471,"(37.60951982027619, 126.83456514393055)",0.0,0,0,NaN


In [ ]:
# 평균 구함
ildo_ilsu[['좌표','DN_avg']].groupby('좌표').agg('mean')

,DN_avg
좌표,
"(37.5785311891648, 126.8649039018377)",NaN
"(37.57857212576646, 126.86716739924644)",NaN
"(37.57859257780594, 126.8682991496677)",NaN
"(37.57869467538848, 126.87395791891903)",NaN
"(37.578715062381775, 126.87508967619401)",NaN
...,...
"(37.746159509215985, 126.91570480030893)",NaN
"(37.746999986854476, 126.91227649424582)",NaN
"(37.74702008221923, 126.91341083764837)",NaN


In [ ]:
godo_350=ildo_ilsu[['좌표','DN_avg']].groupby('좌표').agg('mean').round(0).reset_index()
godo_350.columns=['좌표','DN_avg']
godo_350['DN_avg']=godo_350['DN_avg'].dropna().astype(int)

In [ ]:
godo_350[godo_350['DN_avg'].isna()]

,좌표,DN_avg
0,"(37.5785311891648, 126.8649039018377)",NaN
1,"(37.57857212576646, 126.86716739924644)",NaN
2,"(37.57859257780594, 126.8682991496677)",NaN
3,"(37.57869467538848, 126.87395791891903)",NaN
4,"(37.578715062381775, 126.87508967619401)",NaN
...,...,...
20832,"(37.746159509215985, 126.91570480030893)",NaN
20833,"(37.746999986854476, 126.91227649424582)",NaN
20834,"(37.74702008221923, 126.91341083764837)",NaN
20835,"(37.747040166676285, 126.91454518217792)",NaN


In [ ]:
godo_350

,좌표,DN_avg
0,"(37.5785311891648, 126.8649039018377)",NaN
1,"(37.57857212576646, 126.86716739924644)",NaN
2,"(37.57859257780594, 126.8682991496677)",NaN
3,"(37.57869467538848, 126.87395791891903)",NaN
4,"(37.578715062381775, 126.87508967619401)",NaN
...,...,...
20832,"(37.746159509215985, 126.91570480030893)",NaN
20833,"(37.746999986854476, 126.91227649424582)",NaN
20834,"(37.74702008221923, 126.91341083764837)",NaN
20835,"(37.747040166676285, 126.91454518217792)",NaN


In [ ]:
godo_350.to_csv('./LSCP/전체좌표_godo_300.csv')

# 4. 자전거 도로

In [ ]:
goyang_road

,UFID,재질,자전거도로유/무,종류,geometry
0,TRN0400000001AAEA,SWQ001,BYC001,SWK001,"MULTILINESTRING ((126.87388 37.59902, 126.8739..."
1,TRN0400000001JVQG,SWQ001,BYC001,SWK001,"MULTILINESTRING ((126.87432 37.59904, 126.8743..."
2,TRN0400000001KUOE,SWQ001,BYC001,SWK001,"MULTILINESTRING ((126.87886 37.64444, 126.8792..."
3,TRN0400000001DA87,SWQ001,BYC001,SWK001,"MULTILINESTRING ((126.87393 37.64831, 126.8744..."
4,TRN0400000001DA87,SWQ001,BYC001,SWK001,"MULTILINESTRING ((126.87384 37.64826, 126.8737..."
...,...,...,...,...,...
4380,TRN05000000019YCW,SWQ001,BYC001,SWK002,"MULTILINESTRING ((126.75044 37.68126, 126.7503..."
4381,TRN05000000019YDX,SWQ001,BYC001,SWK002,"MULTILINESTRING ((126.75159 37.68259, 126.7516..."
4382,TRN05000000019YEY,SWQ001,BYC001,SWK002,"MULTILINESTRING ((126.75000 37.68337, 126.7500..."
4383,TRN05000000019YFZ,SWQ001,BYC001,SWK002,"MULTILINESTRING ((126.75115 37.68707, 126.7511..."


In [ ]:
#001이 유
goyang_road['자전거도로유/무'].unique()

array(['BYC001', 'BYC002'], dtype=object)

In [ ]:
def geo_bike(lscp_pop, target):
  lscp=[]
  for sta, lat, long in tqdm(zip(lscp_pop['좌표'],lscp_pop['위도'],lscp_pop['경도'])):
    for nm, geometry in zip(target['자전거도로유/무'],target['geometry']):
      h=[]
      ex=0
      x = (lat, long)
      if nm=='BYC001':
        ex=1
      points = [point for polygon in geometry for point in polygon.coords[:]]
      for t_long, t_lat in points:
        y=(t_lat, t_long)
        h.append(haversine(x,y, unit='m'))
      if min(h)<=300:
        lscp.append([sta,ex,min(h)])
  return lscp

In [ ]:
bike_road=pd.DataFrame(geo_bike(lscp_all,goyang_road),columns=['좌표','Road_yes','Harv_dis'])
bike_road

,좌표,Road_yes,Harv_dis
0,"(37.57961579548528, 126.87506406643347)",1,158.689614
1,"(37.57961579548528, 126.87506406643347)",1,80.159747
2,"(37.57961579548528, 126.87506406643347)",0,47.772169
3,"(37.57961579548528, 126.87506406643347)",0,282.026262
4,"(37.57961579548528, 126.87506406643347)",0,202.927029
...,...,...,...
108,"(37.581437639362306, 126.87614464275404)",0,273.590518
109,"(37.581437639362306, 126.87614464275404)",1,132.565879
110,"(37.581437639362306, 126.87614464275404)",1,173.363648
111,"(37.581949321675445, 126.85461479407492)",1,178.605071


In [ ]:
bike_road_y=bike_road[['좌표','Road_yes']][bike_road['Road_yes']==1].drop_duplicates().reset_index(drop=True)

In [ ]:
lscp_base

,좌표
0,"(37.57961579548528, 126.87506406643347)"
1,"(37.57963617229605, 126.87619583846381)"
2,"(37.57965653826528, 126.87732761163377)"
3,"(37.58049614012486, 126.87390667102693)"
4,"(37.5805165284361, 126.8750384555327)"
...,...
20832,"(37.74702008221923, 126.91341083764837)"
20833,"(37.747040166676285, 126.91454518217792)"
20834,"(37.74706024022562, 126.9156795278339)"
20835,"(37.60951982027619, 126.83456514393055)"


In [ ]:
bike_road_y

,좌표,Road_yes
0,"(37.57961579548528, 126.87506406643347)",1
1,"(37.57963617229605, 126.87619583846381)",1
2,"(37.57965653826528, 126.87732761163377)",1
3,"(37.58049614012486, 126.87390667102693)",1
4,"(37.5805165284361, 126.8750384555327)",1
5,"(37.58053690590551, 126.87617024117878)",1
6,"(37.58055727253304, 126.87730202796448)",1
7,"(37.58106917422839, 126.85577243099591)",1
8,"(37.58108973666921, 126.85690421079029)",1
9,"(37.581110288268384, 126.85803599173475)",1


In [ ]:
bike_100=lscp_base.merge(bike_road_y,on='좌표',how='outer').fillna(0)

In [ ]:
bike_100['Road_yes']=bike_100['Road_yes'].astype(int)

In [ ]:
bike_100

,좌표,Road_yes
0,"(37.57961579548528, 126.87506406643347)",1
1,"(37.57963617229605, 126.87619583846381)",1
2,"(37.57965653826528, 126.87732761163377)",1
3,"(37.58049614012486, 126.87390667102693)",1
4,"(37.5805165284361, 126.8750384555327)",1
...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0
20833,"(37.747040166676285, 126.91454518217792)",0
20834,"(37.74706024022562, 126.9156795278339)",0
20835,"(37.60951982027619, 126.83456514393055)",0


In [ ]:
bike_100.to_csv('./LSCP/전체좌표_bike_100.csv')

# 그 외

#### 데이터 파악 (킨텍스, 호수, 공원)


In [ ]:
fes_space[fes_space['명칭']=='KINTEX']

,명칭,위도,경도,geometry
0,KINTEX,37.669028,126.746184,None


In [ ]:
fes_space['위도']=fes_space['위도'].astype('float64')
fes_space['경도']=fes_space['경도'].astype('float64')


In [ ]:
center = fes_space['위도'].mean(), fes_space['경도'].mean()
zoom = 15
f = folium.Map(location = center, 
               zoom_start = zoom,
               tiles = 'http://api.vworld.kr/req/wmts/1.0.0/FEDE7284-9E9D-3624-BCA7-DDB9767882ED/Base/{z}/{y}/{x}.png',
               attr = '고양시'
              )
for index, row in fes_space.iterrows():
    folium.Marker([row['위도'], row['경도']]).add_to(f)
    folium.Circle([row['위도'], row['경도']],
                  radius = 250, color = 'red' ).add_to(f)
f

KINTEX는 250m를 추가한 거리로,   
호수공원은 도시계획 데이터를 활용하는 쪽으로 (166, 14)    
기타 공원 (27, 28, 45, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 101, 160,161,162, 164, 165, 211, 212, 213, 214, 215, 216, 219, 220, 222, 223, 228, 229, 232, 233, 234, 236, 241, 247, 293, 297, 403,562, 581, 659, 730, 1091,1092,1093,1094)

In [ ]:
physi_curr['GBN'].unique()

array(['육상', '축구장', '수영장', '빙상장', '풋살구장', '배드민턴장', '테니스장', '게이트볼장',
       '파크골프장', '족구장', '인라인하키', '인라인스케이트장', '야구장', '인공암벽장', '농구장', '궁도장',
       '탁구장', '스쿼시', '골프연습장', '헬스장', '클라이밍', '다목적운동장', '야외배드민턴장', '국궁장',
       '실내배드민턴장', 'X게임장', '골프', '다목적시설'], dtype=object)

## Point(X, Y)

In [ ]:
def harv_dis(lscp_pop, target):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('NAME')>=0) | (t.lower().find('station_nm')>=0) | 
                                     (t.lower().find('parking_nm')>=0) | (t.lower().find('place_nm')>=0)|
                                     (t.find('명칭')>=0) for t in target.columns.tolist()].index(True)]
  for sta, lat, long in tqdm(zip(lscp_pop['좌표'],lscp_pop['위도'],lscp_pop['경도'])):
    try:
      for nm, t_lat, t_long in zip(target[target_nm],target['위도'],target['경도']):
        t_lat=float(t_lat)
        t_long=float(t_long)
        x = (lat, long)
        y = (t_lat, t_long) 
        h = haversine(x, y, unit='m')
        if h <=300:
          lscp.append([sta,nm,h])
    except:
      try:
        for nm, t_lat, t_long, t_count in zip(target[target_nm],target['Y'],target['X'],target['count']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=300:
            lscp.append([sta,nm,t_lat,t_long,t_count,h])
      except:
        for nm, t_lat, t_long in zip(target[target_nm],target['Y'],target['X']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=300:
            lscp.append([sta,nm,t_lat,t_long,h])
  return lscp

#### 주차장 (22.주차장정보.csv)

In [ ]:
# 22.주차장정보.csv
parking_info.head()

,parking_nm,road_nm_addr,lot_num_addr,space,X,Y,geometry
0,고양시청 부설주차장,경기도 고양시 덕양구 고양시청로10,경기도 고양시 덕양구 주교동 600,140,126.83207210,37.65804217,None
1,고양제1,경기도 고양시 덕양구 벽제관로 8,경기도 고양시 덕양구 고양동 195-2,94,126.90386130,37.70378261,None
2,고양제2,경기도 고양시 덕양구 혜음로 103,경기도 고양시 덕양구 고양동 444-1,31,126.90534690,37.71021294,None
3,노상 1권역,경기도 고양시 덕양구 고양시청로 10,경기도 고양시 덕양구 주교동 600,97,126.83195970,37.65841450,None
4,노상 3권역,경기도 고양시 덕양구 은빛로 39,경기도 고양시 덕양구 화정동 909,123,126.83217320,37.63809581,None


In [ ]:
parking_info['count']=parking_info['space']

In [ ]:
lscp_parking=pd.DataFrame(harv_dis(lscp_all,parking_info),columns=['좌표','Parking','lat','long','Space_count','Harv_dis'])
lscp_parking.head()

,좌표,Parking,lat,long,Space_count,Harv_dis


In [ ]:
lscp_parking['Space_count']=lscp_parking['Space_count'].astype(int)
bike_parking=lscp_parking[['좌표','Space_count']].groupby('좌표').agg('sum').reset_index()
bike_parking=lscp_base.merge(bike_parking,on='좌표',how='outer').fillna(0)
bike_parking['Space_count']=bike_parking['Space_count'].astype(int)

In [ ]:
bike_parking

,좌표,Space_count
0,"(37.57961579548528, 126.87506406643347)",0
1,"(37.57963617229605, 126.87619583846381)",0
2,"(37.57965653826528, 126.87732761163377)",0
3,"(37.58049614012486, 126.87390667102693)",0
4,"(37.5805165284361, 126.8750384555327)",0
...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0
20833,"(37.747040166676285, 126.91454518217792)",0
20834,"(37.74706024022562, 126.9156795278339)",0
20835,"(37.60951982027619, 126.83456514393055)",0


#### 26.고양시 공연장 박물관 정보.csv

In [ ]:
lscp_cult=pd.DataFrame(harv_dis(lscp_all,theat_muse),columns=['좌표','Culture','lat','long','Harv_dis'])
lscp_cult

,좌표,Culture,lat,long,Harv_dis


In [ ]:
lscp_cult['Culture_count']=1
bike_cult=lscp_cult[['좌표','Culture_count']].groupby('좌표').agg('sum').reset_index()
bike_cult=lscp_base.merge(bike_cult,on='좌표',how='outer').fillna(0)
bike_cult['Culture_count']=bike_cult['Culture_count'].astype(int)
bike_cult

,좌표,Culture_count
0,"(37.57961579548528, 126.87506406643347)",0
1,"(37.57963617229605, 126.87619583846381)",0
2,"(37.57965653826528, 126.87732761163377)",0
3,"(37.58049614012486, 126.87390667102693)",0
4,"(37.5805165284361, 126.8750384555327)",0
...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0
20833,"(37.747040166676285, 126.91454518217792)",0
20834,"(37.74706024022562, 126.9156795278339)",0
20835,"(37.60951982027619, 126.83456514393055)",0


#### 27.고양시 체육시설 현황 정보.csv

In [ ]:
lscp_physi=pd.DataFrame(harv_dis(lscp_all,physi_curr),columns=['좌표','Physical','lat','long','Harv_dis'])
lscp_physi

,좌표,Physical,lat,long,Harv_dis
0,"(37.58106917422839, 126.85577243099591)",대덕야구장,37.579353,126.856932,216.466273
1,"(37.58108973666921, 126.85690421079029)",대덕야구장,37.579353,126.856932,193.132099
2,"(37.581110288268384, 126.85803599173475)",대덕야구장,37.579353,126.856932,218.280667
3,"(37.58199045872832, 126.85687837974507)",대덕야구장,37.579353,126.856932,293.310497


In [ ]:
lscp_physi['Sport_count']=1
bike_physi=lscp_physi[['좌표','Sport_count']].groupby('좌표').agg('sum').reset_index()
bike_physi=lscp_base.merge(bike_physi,on='좌표',how='outer').fillna(0)
bike_physi['Sport_count']=bike_physi['Sport_count'].astype(int)
bike_physi

,좌표,Sport_count
0,"(37.57961579548528, 126.87506406643347)",0
1,"(37.57963617229605, 126.87619583846381)",0
2,"(37.57965653826528, 126.87732761163377)",0
3,"(37.58049614012486, 126.87390667102693)",0
4,"(37.5805165284361, 126.8750384555327)",0
...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0
20833,"(37.747040166676285, 126.91454518217792)",0
20834,"(37.74706024022562, 126.9156795278339)",0
20835,"(37.60951982027619, 126.83456514393055)",0


#### 킨텍스

In [ ]:
lscp_fes=pd.DataFrame(harv_dis(lscp_all,fes_space[fes_space['명칭']=='KINTEX']),columns=['좌표','KINTEX','Harv_dis'])
bike_fes=lscp_base.merge(lscp_fes.iloc[:,:-1],on='좌표',how='outer').fillna(0)
bike_fes['KINTEX'][bike_fes['KINTEX']=='KINTEX']=1

bike_fes

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,좌표,KINTEX
0,"(37.57961579548528, 126.87506406643347)",0
1,"(37.57963617229605, 126.87619583846381)",0
2,"(37.57965653826528, 126.87732761163377)",0
3,"(37.58049614012486, 126.87390667102693)",0
4,"(37.5805165284361, 126.8750384555327)",0
...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0
20833,"(37.747040166676285, 126.91454518217792)",0
20834,"(37.74706024022562, 126.9156795278339)",0
20835,"(37.60951982027619, 126.83456514393055)",0


In [ ]:
bike_fes[bike_fes['KINTEX']==1]

,좌표,KINTEX


#### 데이터 저장

In [ ]:
bike_cult.to_csv('./LSCP/전체좌표_culture_350.csv')
bike_physi.to_csv('./LSCP/전체좌표_physical_350.csv')
bike_parking.to_csv('./LSCP/전체좌표_parking_350.csv')
bike_fes.to_csv('./LSCP/전체좌표_KINTEX_600.csv')

## Multipolygon

In [ ]:
def geo_dis(lscp_pop, target):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('NAME')>=0) | (t.lower().find('dgm_nm')>=0) | 
                                     (t.lower().find('bdtyp_cd')>=0) | (t.lower().find('mnum')>=0) for t in target.columns.tolist()].index(True)]
  for sta, lat, long in tqdm(zip(lscp_pop['좌표'],lscp_pop['위도'],lscp_pop['경도'])):
    for nm, geometry in zip(target[target_nm],target['geometry']):
      h=[]
      x = (lat, long)
      points = [point for polygon in geometry for point in polygon.exterior.coords[:]]
      for t_long, t_lat in points:
        y=(t_lat, t_long)
        h.append(haversine(x,y, unit='m'))
      if min(h)<=300:
        lscp.append([sta,nm,min(h)])
  return lscp

#### 호수 체크

In [ ]:
bike_hosu=pd.DataFrame(geo_dis(lscp_all,space_plan.iloc[[14,166],:]),columns=['좌표','MNUS','Harv_dis'])
bike_hosu['Hosu_yes']=1
bike_hosu=lscp_base.merge(bike_hosu[['좌표','Hosu_yes']],on='좌표',how='outer').fillna(0).drop_duplicates().reset_index(drop=True)
bike_hosu['Hosu_yes']=bike_hosu['Hosu_yes'].astype(int)

In [ ]:
bike_hosu[bike_hosu['Hosu_yes']==1]

,좌표,Hosu_yes


In [ ]:
bike_hosu

,좌표,Hosu_yes
0,"(37.57961579548528, 126.87506406643347)",0
1,"(37.57963617229605, 126.87619583846381)",0
2,"(37.57965653826528, 126.87732761163377)",0
3,"(37.58049614012486, 126.87390667102693)",0
4,"(37.5805165284361, 126.8750384555327)",0
...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0
20833,"(37.747040166676285, 126.91454518217792)",0
20834,"(37.74706024022562, 126.9156795278339)",0
20835,"(37.60951982027619, 126.83456514393055)",0


#### 공원개수

In [ ]:
bike_park=pd.DataFrame(geo_dis(lscp_all,space_plan.iloc[[27, 28, 45, 53, 54, 55, 56, 57, 58, 59, 60, 61, 
                                                             63, 101, 160, 161, 162, 164, 165, 211, 212, 213, 
                                                             214, 215, 216, 219, 220, 222, 223, 228, 229, 232, 
                                                             233, 234, 236, 241, 247, 293, 297, 403,562, 581, 659, 730, 1091,1092,1093],:]),columns=['좌표','MNUS','Harv_dis'])
bike_park['Park_count']=1
# 아래는 유무만 판별하는 코드
# bike_park=bike_park[['Bike_Station','Park_yes']].drop_duplicates()
# 아래는 개수 더하는 코드
bike_park=bike_park[['좌표','Park_count']].groupby('좌표').agg('sum').reset_index()
bike_park.columns=['좌표','Park_count']
bike_park=lscp_base.merge(bike_park[['좌표','Park_count']],on='좌표',how='outer').fillna(0)
bike_park['Park_count']=bike_park['Park_count'].astype(int)

In [ ]:
bike_park['Park_count'].unique()

array([0])

In [ ]:
bike_park

,좌표,Park_count
0,"(37.57961579548528, 126.87506406643347)",0
1,"(37.57963617229605, 126.87619583846381)",0
2,"(37.57965653826528, 126.87732761163377)",0
3,"(37.58049614012486, 126.87390667102693)",0
4,"(37.5805165284361, 126.8750384555327)",0
...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0
20833,"(37.747040166676285, 126.91454518217792)",0
20834,"(37.74706024022562, 126.9156795278339)",0
20835,"(37.60951982027619, 126.83456514393055)",0


In [ ]:
bike_hosu.to_csv('./LSCP/전체좌표_hosu_300.csv')
bike_park.to_csv('./LSCP/전체좌표_park_300.csv')

# 도로명주소

In [ ]:
def geodis2(lscp_pop, target, lscp_distance = 300):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('NAME')>=0) | (t.lower().find('dgm_nm')>=0) | 
                                     (t.lower().find('bdtyp_cd')>=0) | (t.lower().find('mnum')>=0) for t in target.columns.tolist()].index(True)]
  for sta, lat, long in tqdm(zip(lscp_pop['좌표'],lscp_pop['위도'],lscp_pop['경도'])):
    for nm, geometry in zip(target[target_nm],target['geometry']):
      h=[]
      x = (lat, long)
      points = [point for polygon in geometry for point in polygon.exterior.coords[:]]
      for t_long, t_lat in points:
        y=(t_lat, t_long)
        h.append(haversine(x,y, unit='m'))
      if min(h)<=lscp_distance:
        lscp.append([sta,nm,min(h)])
  lscp_pd=pd.DataFrame(lscp,columns=['좌표','BDTYP_CD','Harv_Dis'])
  lscp_pd['BDTYP_CD_count']=1
  lscp_pd=lscp_pd[['좌표','BDTYP_CD_count']].groupby('좌표').agg('sum').reset_index()
  lscp_pd.columns=['좌표','BDTYP_CD_count']
  lscp_pd['좌표'] = lscp_pd['좌표'].astype(str)
  lscp_pd=lscp_base.merge(lscp_pd[['좌표','BDTYP_CD_count']],on='좌표',how='outer').fillna(0)
  lscp_pd['BDTYP_CD_count']=lscp_pd['BDTYP_CD_count'].astype(int)
  return lscp_pd

In [ ]:
# 학습시킬 도로명 코드 
도로명 = ['build_01003',
'build_02003',
'build_03999',
'build_03005',
'build_01001',
'build_13100',
'build_16006',
'build_03001',
'build_17007',
'build_16010',
'build_04001',
'build_03104',
'build_14999',
'build_04005',
'build_04010',
'build_04004',
'build_14001',
'build_04999',
'build_19003',
'build_04006',
'build_02002',
'build_01002',
'build_04402',
'build_04403',
'build_08102',
'build_17999',
'build_10202',
'build_17101',
'build_17203',
'build_03020',
'build_19007',
'build_04299',
'build_08201',
'build_16001',
'build_06999',
'build_03021',
'build_04199',
'build_05107',
'build_08003',
'build_08101',
'build_15001',
'build_06203',
'build_08300',
'build_02001',
'build_04499',
'build_17299',
'build_05999',
'build_07107',
'build_16002',
'build_03199',
'build_04102',
'build_03007',
'build_03002',
'build_18003',
'build_05101',
'build_05199',
'build_19005',
'build_08999',
'build_15005',
'build_01004',
'build_04002',
'build_18002',
'build_16005',
'build_90001',
'build_05599',
'build_06301',
'build_03012',
'build_08203',
'build_04018',
'build_04007',
'build_09999',
'build_02007',
'build_11102',
'build_17204',
'build_08004',
'build_04014',
'build_11201',
'build_05499',
'build_08005',
'build_15002',
'build_04201',
'build_07999',
'build_09004',
'build_17004',
'build_17005',
'build_08800',
'build_04012',
'build_03107',
'build_02004',
'build_02006',
'build_02005',
'build_03015',
'build_04401',
'build_10102',
'build_03008',
'build_08400',
'build_10299',
'build_16009',
'build_10204',
'build_16999',
'build_10199',
'build_03010',
'build_03004',
'build_19999',
'build_08500',
'build_05103',
'build_08602',
'build_21999',
'build_07105',
'build_07302',
'build_04003',
'build_14002',
'build_03006',
'build_17003',
'build_04017',
'build_07104',
'build_03103',
'build_03101',
'build_08103',
'build_03105',
'build_03100',
'build_05109',
'build_15009',
'build_18999',
'build_20003',
'build_18004',
'build_17207',
'build_11199',
'build_04009',
'build_04505',
'build_06100',
'build_03003',
'build_10201',
'build_06299',
'build_04101',
'build_08104',
'build_16008',
'build_08202',
'build_05501',
'build_11101',
'build_04020',
'build_04203',
'build_16004',
'build_16003',
'build_03009',
'build_04105',
'build_08199',
'build_06305',
'build_08204',
'build_03022',
'build_17104',
'build_05102',
'build_03013',
'build_03023',
'build_20999',
'build_07108',
'build_21006',
'build_05108',
'build_08001',
'build_08299',
'build_04303',
'build_06310',
'build_05106',
'build_03102',
'build_06205',
'build_09001',
'build_17006',
'build_05105',
'build_05299',
'build_05404',
'build_04016',
'build_18001',
'build_03011',
'build_05104',
'build_03108',
'build_07199',
'build_06202',
'build_03016',
'build_04202',
'build_19004',
'build_03014',
'build_12999',
'build_05301',
'build_13200',
'build_19006',
'build_04107',
'build_07101',
'build_06204',
'build_04022',
'build_07102',
'build_06303',
'build_15006',
'build_08601',
'build_15999',
'build_04405',
'build_16007',
'build_03019',
'build_03017',
'build_03018',
'build_04023',
'build_11299',
'build_04008',
'build_09005',
'build_03106',
'build_05303',
'build_05699',
'build_04399',
'build_05403',
'build_15007',
'build_05505',
'build_04021',
'build_09011',
'build_20002',
'build_06201',
'build_17105',
'build_08699',
'build_03109',
'build_27999',
'build_90002',
'build_05502']

In [ ]:
# 건물용도코드 불러오기 
address_code = address_code[581:].reset_index().drop('index', axis = 1)
address_code.columns = ['code', 'code_name']
address_code

,code,code_name
0,01000,단독주택
1,01001,단독주택
2,01002,다중주택
3,01003,다가구주택
4,01004,공관
...,...,...
319,21003,어린이회관
320,21004,관망탑
321,21005,휴게소
322,21006,관광지시설


In [ ]:
# 좌표당 해당 도로명 주소코드에 해당하는 건물 개수 세고 저장하는 명령문 출력
fail = []
for x in 도로명:
  if x[6:] not in address_code['code'].values:
    print(f'print("{x}")')
    print(f"{x}=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='{x[6:]}'])")
    print(f"{x}.columns = ['좌표', 'CD_{x[6:]}']") 
    print(f'{x}.to_csv("./LSCP/전체좌표_{x}.csv", index = False)')
    print('\n')

  for code, code_name in zip(address_code['code'], address_code['code_name']):
    if x[6:] == code:
      print(f'print("{x}")')
      print(f"{x}=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='{x[6:]}'])")
      print(f"{x}.columns = ['좌표', 'CD_{x[6:]}_{code_name}']")  
      print(f'{x}.to_csv("./LSCP/전체좌표_{x}.csv", index = False)')
      print('\n')    

print("build_01003")
build_01003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='01003'])
build_01003.columns = ['좌표', 'CD_01003_다가구주택']
build_01003.to_csv("./LSCP/전체좌표_build_01003.csv", index = False)


print("build_02003")
build_02003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='02003'])
build_02003.columns = ['좌표', 'CD_02003_다세대주택']
build_02003.to_csv("./LSCP/전체좌표_build_02003.csv", index = False)


print("build_03999")
build_03999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03999'])
build_03999.columns = ['좌표', 'CD_03999_기타제1종근생']
build_03999.to_csv("./LSCP/전체좌표_build_03999.csv", index = False)


print("build_03005")
build_03005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03005'])
build_03005.columns = ['좌표', 'CD_03005_의원']
build_03005.to_csv("./LSCP/전체좌표_build_03005.csv", index = False)


print("build_01001")
build_01001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='01001'])
build_01001.columns = ['좌표', 'CD_01001_단독주택']
build_01001.to_csv("./LS

In [ ]:
# 좌표당 해당 도로명 주소코드에 해당하는 건물 개수 세고 저장하는 명령문 실행 

print("build_01003")
build_01003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='01003'])
build_01003.columns = ['좌표', 'CD_01003_다가구주택']
build_01003.to_csv("./LSCP/전체좌표_build_01003.csv", index = False)


print("build_02003")
build_02003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='02003'])
build_02003.columns = ['좌표', 'CD_02003_다세대주택']
build_02003.to_csv("./LSCP/전체좌표_build_02003.csv", index = False)


print("build_03999")
build_03999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03999'])
build_03999.columns = ['좌표', 'CD_03999_기타제1종근생']
build_03999.to_csv("./LSCP/전체좌표_build_03999.csv", index = False)


print("build_03005")
build_03005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03005'])
build_03005.columns = ['좌표', 'CD_03005_의원']
build_03005.to_csv("./LSCP/전체좌표_build_03005.csv", index = False)


print("build_01001")
build_01001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='01001'])
build_01001.columns = ['좌표', 'CD_01001_단독주택']
build_01001.to_csv("./LSCP/전체좌표_build_01001.csv", index = False)


print("build_13100")
build_13100=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='13100'])
build_13100.columns = ['좌표', 'CD_13100_일반공장']
build_13100.to_csv("./LSCP/전체좌표_build_13100.csv", index = False)


print("build_16006")
build_16006=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16006'])
build_16006.columns = ['좌표', 'CD_16006_정비공장']
build_16006.to_csv("./LSCP/전체좌표_build_16006.csv", index = False)


print("build_03001")
build_03001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03001'])
build_03001.columns = ['좌표', 'CD_03001_소매점']
build_03001.to_csv("./LSCP/전체좌표_build_03001.csv", index = False)


print("build_17007")
build_17007=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17007'])
build_17007.columns = ['좌표', 'CD_17007_온실']
build_17007.to_csv("./LSCP/전체좌표_build_17007.csv", index = False)


print("build_16010")
build_16010=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16010'])
build_16010.columns = ['좌표', 'CD_16010_주기장']
build_16010.to_csv("./LSCP/전체좌표_build_16010.csv", index = False)


print("build_04001")
build_04001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04001'])
build_04001.columns = ['좌표', 'CD_04001_일반음식점']
build_04001.to_csv("./LSCP/전체좌표_build_04001.csv", index = False)


print("build_03104")
build_03104=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03104'])
build_03104.columns = ['좌표', 'CD_03104_소방서']
build_03104.to_csv("./LSCP/전체좌표_build_03104.csv", index = False)


print("build_14999")
build_14999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='14999'])
build_14999.columns = ['좌표', 'CD_14999_기타창고시설']
build_14999.to_csv("./LSCP/전체좌표_build_14999.csv", index = False)


print("build_04005")
build_04005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04005'])
build_04005.columns = ['좌표', 'CD_04005_제조업소']
build_04005.to_csv("./LSCP/전체좌표_build_04005.csv", index = False)


print("build_04010")
build_04010=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04010'])
build_04010.columns = ['좌표', 'CD_04010_학원']
build_04010.to_csv("./LSCP/전체좌표_build_04010.csv", index = False)


print("build_04004")
build_04004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04004'])
build_04004.columns = ['좌표', 'CD_04004_서점']
build_04004.to_csv("./LSCP/전체좌표_build_04004.csv", index = False)


print("build_14001")
build_14001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='14001'])
build_14001.columns = ['좌표', 'CD_14001_창고']
build_14001.to_csv("./LSCP/전체좌표_build_14001.csv", index = False)


print("build_04999")
build_04999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04999'])
build_04999.columns = ['좌표', 'CD_04999_기타제2종근생']
build_04999.to_csv("./LSCP/전체좌표_build_04999.csv", index = False)


print("build_19003")
build_19003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='19003'])
build_19003.columns = ['좌표', 'CD_19003_군사시설']
build_19003.to_csv("./LSCP/전체좌표_build_19003.csv", index = False)


print("build_04006")
build_04006=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04006'])
build_04006.columns = ['좌표', 'CD_04006_수리점']
build_04006.to_csv("./LSCP/전체좌표_build_04006.csv", index = False)


print("build_02002")
build_02002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='02002'])
build_02002.columns = ['좌표', 'CD_02002_연립주택']
build_02002.to_csv("./LSCP/전체좌표_build_02002.csv", index = False)


print("build_01002")
build_01002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='01002'])
build_01002.columns = ['좌표', 'CD_01002_다중주택']
build_01002.to_csv("./LSCP/전체좌표_build_01002.csv", index = False)


print("build_04402")
build_04402=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04402'])
build_04402.columns = ['좌표', 'CD_04402_사무소']
build_04402.to_csv("./LSCP/전체좌표_build_04402.csv", index = False)


print("build_04403")
build_04403=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04403'])
build_04403.columns = ['좌표', 'CD_04403_부동산중개업소']
build_04403.to_csv("./LSCP/전체좌표_build_04403.csv", index = False)


print("build_08102")
build_08102=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08102'])
build_08102.columns = ['좌표', 'CD_08102_중학교']
build_08102.to_csv("./LSCP/전체좌표_build_08102.csv", index = False)


print("build_17999")
build_17999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17999'])
build_17999.columns = ['좌표', 'CD_17999_기타동식물관련시설']
build_17999.to_csv("./LSCP/전체좌표_build_17999.csv", index = False)


print("build_10202")
build_10202=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='10202'])
build_10202.columns = ['좌표', 'CD_10202_오피스텔']
build_10202.to_csv("./LSCP/전체좌표_build_10202.csv", index = False)


print("build_17101")
build_17101=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17101'])
build_17101.columns = ['좌표', 'CD_17101_축사']
build_17101.to_csv("./LSCP/전체좌표_build_17101.csv", index = False)


print("build_17203")
build_17203=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17203'])
build_17203.columns = ['좌표', 'CD_17203_관리사']
build_17203.to_csv("./LSCP/전체좌표_build_17203.csv", index = False)


print("build_03020")
build_03020=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03020'])
build_03020.columns = ['좌표', 'CD_03020_체육도장']
build_03020.to_csv("./LSCP/전체좌표_build_03020.csv", index = False)


print("build_19007")
build_19007=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='19007'])
build_19007.columns = ['좌표', 'CD_19007_촬영소']
build_19007.to_csv("./LSCP/전체좌표_build_19007.csv", index = False)


print("build_04299")
build_04299=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04299'])
build_04299.columns = ['좌표', 'CD_04299_기타종교집회장']
build_04299.to_csv("./LSCP/전체좌표_build_04299.csv", index = False)


print("build_08201")
build_08201=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08201'])
build_08201.columns = ['좌표', 'CD_08201_유치원']
build_08201.to_csv("./LSCP/전체좌표_build_08201.csv", index = False)


print("build_16001")
build_16001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16001'])
build_16001.columns = ['좌표', 'CD_16001_주차장']
build_16001.to_csv("./LSCP/전체좌표_build_16001.csv", index = False)


print("build_06999")
build_06999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06999'])
build_06999.columns = ['좌표', 'CD_06999_기타판매및영업시설']
build_06999.to_csv("./LSCP/전체좌표_build_06999.csv", index = False)


print("build_03021")
build_03021=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03021'])
build_03021.columns = ['좌표', 'CD_03021_마을공회당']
build_03021.to_csv("./LSCP/전체좌표_build_03021.csv", index = False)


print("build_04199")
build_04199=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04199'])
build_04199.columns = ['좌표', 'CD_04199_기타운동시설']
build_04199.to_csv("./LSCP/전체좌표_build_04199.csv", index = False)


print("build_05107")
build_05107=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05107'])
build_05107.columns = ['좌표', 'CD_05107_제실']
build_05107.to_csv("./LSCP/전체좌표_build_05107.csv", index = False)


print("build_08003")
build_08003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08003'])
build_08003.columns = ['좌표', 'CD_08003_학원']
build_08003.to_csv("./LSCP/전체좌표_build_08003.csv", index = False)


print("build_08003")
build_08003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08003'])
build_08003.columns = ['좌표', 'CD_08003_어린이집']
build_08003.to_csv("./LSCP/전체좌표_build_08003.csv", index = False)


print("build_08101")
build_08101=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08101'])
build_08101.columns = ['좌표', 'CD_08101_초등학교']
build_08101.to_csv("./LSCP/전체좌표_build_08101.csv", index = False)


print("build_15001")
build_15001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='15001'])
build_15001.columns = ['좌표', 'CD_15001_주유소']
build_15001.to_csv("./LSCP/전체좌표_build_15001.csv", index = False)


print("build_06203")
build_06203=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06203'])
build_06203.columns = ['좌표', 'CD_06203_대형판매점']
build_06203.to_csv("./LSCP/전체좌표_build_06203.csv", index = False)


print("build_08300")
build_08300=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08300'])
build_08300.columns = ['좌표', 'CD_08300_노인복지시설']
build_08300.to_csv("./LSCP/전체좌표_build_08300.csv", index = False)


print("build_02001")
build_02001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='02001'])
build_02001.columns = ['좌표', 'CD_02001_아파트']
build_02001.to_csv("./LSCP/전체좌표_build_02001.csv", index = False)


print("build_04499")
build_04499=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04499'])
build_04499.columns = ['좌표', 'CD_04499_기타사무소']
build_04499.to_csv("./LSCP/전체좌표_build_04499.csv", index = False)


print("build_17299")
build_17299=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17299'])
build_17299.columns = ['좌표', 'CD_17299_기타가축시설']
build_17299.to_csv("./LSCP/전체좌표_build_17299.csv", index = False)


print("build_05999")
build_05999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05999'])
build_05999.columns = ['좌표', 'CD_05999_기타문화및집회시설']
build_05999.to_csv("./LSCP/전체좌표_build_05999.csv", index = False)


print("build_07107")
build_07107=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='07107'])
build_07107.columns = ['좌표', 'CD_07107_병원']
build_07107.to_csv("./LSCP/전체좌표_build_07107.csv", index = False)


print("build_16002")
build_16002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16002'])
build_16002.columns = ['좌표', 'CD_16002_세차장']
build_16002.to_csv("./LSCP/전체좌표_build_16002.csv", index = False)


print("build_03199")
build_03199=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03199'])
build_03199.columns = ['좌표', 'CD_03199_기타공공시설']
build_03199.to_csv("./LSCP/전체좌표_build_03199.csv", index = False)


print("build_04102")
build_04102=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04102'])
build_04102.columns = ['좌표', 'CD_04102_체력단련장']
build_04102.to_csv("./LSCP/전체좌표_build_04102.csv", index = False)


print("build_03007")
build_03007=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03007'])
build_03007.columns = ['좌표', 'CD_03007_마을공동시설']
build_03007.to_csv("./LSCP/전체좌표_build_03007.csv", index = False)


print("build_03002")
build_03002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03002'])
build_03002.columns = ['좌표', 'CD_03002_휴게음식점']
build_03002.to_csv("./LSCP/전체좌표_build_03002.csv", index = False)


print("build_18003")
build_18003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='18003'])
build_18003.columns = ['좌표', 'CD_18003_폐기물재활용시설']
build_18003.to_csv("./LSCP/전체좌표_build_18003.csv", index = False)


print("build_05101")
build_05101=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05101'])
build_05101.columns = ['좌표', 'CD_05101_교회']
build_05101.to_csv("./LSCP/전체좌표_build_05101.csv", index = False)


print("build_05199")
build_05199=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05199'])
build_05199.columns = ['좌표', 'CD_05199_기타종교집회장']
build_05199.to_csv("./LSCP/전체좌표_build_05199.csv", index = False)


print("build_19005")
build_19005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='19005'])
build_19005.columns = ['좌표', 'CD_19005_방송국']
build_19005.to_csv("./LSCP/전체좌표_build_19005.csv", index = False)


print("build_08999")
build_08999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08999'])
build_08999.columns = ['좌표', 'CD_08999_기타교육연구및복지시설']
build_08999.to_csv("./LSCP/전체좌표_build_08999.csv", index = False)


print("build_15005")
build_15005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='15005'])
build_15005.columns = ['좌표', 'CD_15005_액화가스취급소']
build_15005.to_csv("./LSCP/전체좌표_build_15005.csv", index = False)


print("build_01004")
build_01004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='01004'])
build_01004.columns = ['좌표', 'CD_01004_공관']
build_01004.to_csv("./LSCP/전체좌표_build_01004.csv", index = False)


print("build_04002")
build_04002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04002'])
build_04002.columns = ['좌표', 'CD_04002_휴게음식점']
build_04002.to_csv("./LSCP/전체좌표_build_04002.csv", index = False)


print("build_18002")
build_18002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='18002'])
build_18002.columns = ['좌표', 'CD_18002_폐기물처리시설']
build_18002.to_csv("./LSCP/전체좌표_build_18002.csv", index = False)


print("build_16005")
build_16005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16005'])
build_16005.columns = ['좌표', 'CD_16005_매매장']
build_16005.to_csv("./LSCP/전체좌표_build_16005.csv", index = False)


print("build_90001")
build_90001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='90001'])
build_90001.columns = ['좌표', 'CD_90001']
build_90001.to_csv("./LSCP/전체좌표_build_90001.csv", index = False)


print("build_05599")
build_05599=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05599'])
build_05599.columns = ['좌표', 'CD_05599_기타전시장']
build_05599.to_csv("./LSCP/전체좌표_build_05599.csv", index = False)


print("build_06301")
build_06301=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06301'])
build_06301.columns = ['좌표', 'CD_06301_상점']
build_06301.to_csv("./LSCP/전체좌표_build_06301.csv", index = False)


print("build_03012")
build_03012=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03012'])
build_03012.columns = ['좌표', 'CD_03012_공중화장실']
build_03012.to_csv("./LSCP/전체좌표_build_03012.csv", index = False)


print("build_08203")
build_08203=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08203'])
build_08203.columns = ['좌표', 'CD_08203']
build_08203.to_csv("./LSCP/전체좌표_build_08203.csv", index = False)


print("build_04018")
build_04018=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04018'])
build_04018.columns = ['좌표', 'CD_04018_자동차영업소']
build_04018.to_csv("./LSCP/전체좌표_build_04018.csv", index = False)


print("build_04007")
build_04007=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04007'])
build_04007.columns = ['좌표', 'CD_04007_게임제공업소']
build_04007.to_csv("./LSCP/전체좌표_build_04007.csv", index = False)


print("build_09999")
build_09999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='09999'])
build_09999.columns = ['좌표', 'CD_09999_기타운동시설']
build_09999.to_csv("./LSCP/전체좌표_build_09999.csv", index = False)


print("build_02007")
build_02007=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='02007'])
build_02007.columns = ['좌표', 'CD_02007_기숙사']
build_02007.to_csv("./LSCP/전체좌표_build_02007.csv", index = False)


print("build_11102")
build_11102=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='11102'])
build_11102.columns = ['좌표', 'CD_11102_여관']
build_11102.to_csv("./LSCP/전체좌표_build_11102.csv", index = False)


print("build_17204")
build_17204=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17204'])
build_17204.columns = ['좌표', 'CD_17204_가축용창고']
build_17204.to_csv("./LSCP/전체좌표_build_17204.csv", index = False)


print("build_08004")
build_08004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08004'])
build_08004.columns = ['좌표', 'CD_08004_연구소']
build_08004.to_csv("./LSCP/전체좌표_build_08004.csv", index = False)


print("build_04014")
build_04014=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04014'])
build_04014.columns = ['좌표', 'CD_04014_독서실']
build_04014.to_csv("./LSCP/전체좌표_build_04014.csv", index = False)


print("build_11201")
build_11201=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='11201'])
build_11201.columns = ['좌표', 'CD_11201_관광호텔']
build_11201.to_csv("./LSCP/전체좌표_build_11201.csv", index = False)


print("build_05499")
build_05499=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05499'])
build_05499.columns = ['좌표', 'CD_05499_기타관람장']
build_05499.to_csv("./LSCP/전체좌표_build_05499.csv", index = False)


print("build_08005")
build_08005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08005'])
build_08005.columns = ['좌표', 'CD_08005_도서관']
build_08005.to_csv("./LSCP/전체좌표_build_08005.csv", index = False)


print("build_15002")
build_15002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='15002'])
build_15002.columns = ['좌표', 'CD_15002_액화석유가스충전소']
build_15002.to_csv("./LSCP/전체좌표_build_15002.csv", index = False)


print("build_04201")
build_04201=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04201'])
build_04201.columns = ['좌표', 'CD_04201_교회']
build_04201.to_csv("./LSCP/전체좌표_build_04201.csv", index = False)


print("build_07999")
build_07999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='07999'])
build_07999.columns = ['좌표', 'CD_07999_기타의료시설']
build_07999.to_csv("./LSCP/전체좌표_build_07999.csv", index = False)


print("build_09004")
build_09004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='09004'])
build_09004.columns = ['좌표', 'CD_09004_체육도장']
build_09004.to_csv("./LSCP/전체좌표_build_09004.csv", index = False)


print("build_17004")
build_17004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17004'])
build_17004.columns = ['좌표', 'CD_17004_도계장']
build_17004.to_csv("./LSCP/전체좌표_build_17004.csv", index = False)


print("build_17005")
build_17005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17005'])
build_17005.columns = ['좌표', 'CD_17005_버섯재배사']
build_17005.to_csv("./LSCP/전체좌표_build_17005.csv", index = False)


print("build_08800")
build_08800=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08800'])
build_08800.columns = ['좌표', 'CD_08800']
build_08800.to_csv("./LSCP/전체좌표_build_08800.csv", index = False)


print("build_04012")
build_04012=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04012'])
build_04012.columns = ['좌표', 'CD_04012_동물병원']
build_04012.to_csv("./LSCP/전체좌표_build_04012.csv", index = False)


print("build_03107")
build_03107=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03107'])
build_03107.columns = ['좌표', 'CD_03107_방송국']
build_03107.to_csv("./LSCP/전체좌표_build_03107.csv", index = False)


print("build_02004")
build_02004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='02004'])
build_02004.columns = ['좌표', 'CD_02004_생활편익시설']
build_02004.to_csv("./LSCP/전체좌표_build_02004.csv", index = False)


print("build_02006")
build_02006=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='02006'])
build_02006.columns = ['좌표', 'CD_02006_복리시설']
build_02006.to_csv("./LSCP/전체좌표_build_02006.csv", index = False)


print("build_02005")
build_02005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='02005'])
build_02005.columns = ['좌표', 'CD_02005_부대시설']
build_02005.to_csv("./LSCP/전체좌표_build_02005.csv", index = False)


print("build_03015")
build_03015=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03015'])
build_03015.columns = ['좌표', 'CD_03015_한의원']
build_03015.to_csv("./LSCP/전체좌표_build_03015.csv", index = False)


print("build_04401")
build_04401=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04401'])
build_04401.columns = ['좌표', 'CD_04401_금융업소']
build_04401.to_csv("./LSCP/전체좌표_build_04401.csv", index = False)


print("build_10102")
build_10102=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='10102'])
build_10102.columns = ['좌표', 'CD_10102_자치단체청사']
build_10102.to_csv("./LSCP/전체좌표_build_10102.csv", index = False)


print("build_03008")
build_03008=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03008'])
build_03008.columns = ['좌표', 'CD_03008_변전소']
build_03008.to_csv("./LSCP/전체좌표_build_03008.csv", index = False)


print("build_08400")
build_08400=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08400'])
build_08400.columns = ['좌표', 'CD_08400_사회복지시설']
build_08400.to_csv("./LSCP/전체좌표_build_08400.csv", index = False)


print("build_10299")
build_10299=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='10299'])
build_10299.columns = ['좌표', 'CD_10299_기타일반업무시설']
build_10299.to_csv("./LSCP/전체좌표_build_10299.csv", index = False)


print("build_16009")
build_16009=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16009'])
build_16009.columns = ['좌표', 'CD_16009_차고']
build_16009.to_csv("./LSCP/전체좌표_build_16009.csv", index = False)


print("build_10204")
build_10204=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='10204'])
build_10204.columns = ['좌표', 'CD_10204_사무소']
build_10204.to_csv("./LSCP/전체좌표_build_10204.csv", index = False)


print("build_16999")
build_16999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16999'])
build_16999.columns = ['좌표', 'CD_16999_기타자동차관련시설']
build_16999.to_csv("./LSCP/전체좌표_build_16999.csv", index = False)


print("build_10199")
build_10199=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='10199'])
build_10199.columns = ['좌표', 'CD_10199_기타공공업무시설']
build_10199.to_csv("./LSCP/전체좌표_build_10199.csv", index = False)


print("build_03010")
build_03010=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03010'])
build_03010.columns = ['좌표', 'CD_03010_정수장']
build_03010.to_csv("./LSCP/전체좌표_build_03010.csv", index = False)


print("build_03004")
build_03004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03004'])
build_03004.columns = ['좌표', 'CD_03004_일반목욕장']
build_03004.to_csv("./LSCP/전체좌표_build_03004.csv", index = False)


print("build_19999")
build_19999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='19999'])
build_19999.columns = ['좌표', 'CD_19999_기타공공용시설']
build_19999.to_csv("./LSCP/전체좌표_build_19999.csv", index = False)


print("build_08500")
build_08500=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08500'])
build_08500.columns = ['좌표', 'CD_08500_근로복지시설']
build_08500.to_csv("./LSCP/전체좌표_build_08500.csv", index = False)


print("build_05103")
build_05103=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05103'])
build_05103.columns = ['좌표', 'CD_05103_사찰']
build_05103.to_csv("./LSCP/전체좌표_build_05103.csv", index = False)


print("build_08602")
build_08602=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08602'])
build_08602.columns = ['좌표', 'CD_08602_유스호스텔']
build_08602.to_csv("./LSCP/전체좌표_build_08602.csv", index = False)


print("build_21999")
build_21999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='21999'])
build_21999.columns = ['좌표', 'CD_21999_기타관광휴게시설']
build_21999.to_csv("./LSCP/전체좌표_build_21999.csv", index = False)


print("build_07105")
build_07105=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='07105'])
build_07105.columns = ['좌표', 'CD_07105_정신병원']
build_07105.to_csv("./LSCP/전체좌표_build_07105.csv", index = False)


print("build_07302")
build_07302=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='07302'])
build_07302.columns = ['좌표', 'CD_07302_마약진료소']
build_07302.to_csv("./LSCP/전체좌표_build_07302.csv", index = False)


print("build_04003")
build_04003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04003'])
build_04003.columns = ['좌표', 'CD_04003_기원']
build_04003.to_csv("./LSCP/전체좌표_build_04003.csv", index = False)


print("build_14002")
build_14002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='14002'])
build_14002.columns = ['좌표', 'CD_14002_하역장']
build_14002.to_csv("./LSCP/전체좌표_build_14002.csv", index = False)


print("build_03006")
build_03006=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03006'])
build_03006.columns = ['좌표', 'CD_03006_체육장']
build_03006.to_csv("./LSCP/전체좌표_build_03006.csv", index = False)


print("build_17003")
build_17003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17003'])
build_17003.columns = ['좌표', 'CD_17003_도축장']
build_17003.to_csv("./LSCP/전체좌표_build_17003.csv", index = False)


print("build_04017")
build_04017=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04017'])
build_04017.columns = ['좌표', 'CD_04017_의약품도매점']
build_04017.to_csv("./LSCP/전체좌표_build_04017.csv", index = False)


print("build_07104")
build_07104=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='07104'])
build_07104.columns = ['좌표', 'CD_07104_한방병원']
build_07104.to_csv("./LSCP/전체좌표_build_07104.csv", index = False)


print("build_03103")
build_03103=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03103'])
build_03103.columns = ['좌표', 'CD_03103_파출소']
build_03103.to_csv("./LSCP/전체좌표_build_03103.csv", index = False)


print("build_03101")
build_03101=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03101'])
build_03101.columns = ['좌표', 'CD_03101_동사무소']
build_03101.to_csv("./LSCP/전체좌표_build_03101.csv", index = False)


print("build_08103")
build_08103=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08103'])
build_08103.columns = ['좌표', 'CD_08103_고등학교']
build_08103.to_csv("./LSCP/전체좌표_build_08103.csv", index = False)


print("build_03105")
build_03105=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03105'])
build_03105.columns = ['좌표', 'CD_03105_우체국']
build_03105.to_csv("./LSCP/전체좌표_build_03105.csv", index = False)


print("build_03100")
build_03100=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03100'])
build_03100.columns = ['좌표', 'CD_03100_공공시설']
build_03100.to_csv("./LSCP/전체좌표_build_03100.csv", index = False)


print("build_05109")
build_05109=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05109'])
build_05109.columns = ['좌표', 'CD_05109_납골당']
build_05109.to_csv("./LSCP/전체좌표_build_05109.csv", index = False)


print("build_15009")
build_15009=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='15009'])
build_15009.columns = ['좌표', 'CD_15009_석유판매소']
build_15009.to_csv("./LSCP/전체좌표_build_15009.csv", index = False)


print("build_18999")
build_18999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='18999'])
build_18999.columns = ['좌표', 'CD_18999_기타분뇨쓰레기처리시설']
build_18999.to_csv("./LSCP/전체좌표_build_18999.csv", index = False)


print("build_20003")
build_20003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='20003'])
build_20003.columns = ['좌표', 'CD_20003_묘지에 부수되는 건축물']
build_20003.to_csv("./LSCP/전체좌표_build_20003.csv", index = False)


print("build_18004")
build_18004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='18004'])
build_18004.columns = ['좌표', 'CD_18004_고물상']
build_18004.to_csv("./LSCP/전체좌표_build_18004.csv", index = False)


print("build_17207")
build_17207=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17207'])
build_17207.columns = ['좌표', 'CD_17207_실험동물사육시설']
build_17207.to_csv("./LSCP/전체좌표_build_17207.csv", index = False)


print("build_11199")
build_11199=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='11199'])
build_11199.columns = ['좌표', 'CD_11199_기타일반숙박시설']
build_11199.to_csv("./LSCP/전체좌표_build_11199.csv", index = False)


print("build_04009")
build_04009=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04009'])
build_04009.columns = ['좌표', 'CD_04009_표구점']
build_04009.to_csv("./LSCP/전체좌표_build_04009.csv", index = False)


print("build_04505")
build_04505=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04505'])
build_04505.columns = ['좌표', 'CD_04505']
build_04505.to_csv("./LSCP/전체좌표_build_04505.csv", index = False)


print("build_06100")
build_06100=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06100'])
build_06100.columns = ['좌표', 'CD_06100_도매시장']
build_06100.to_csv("./LSCP/전체좌표_build_06100.csv", index = False)


print("build_03003")
build_03003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03003'])
build_03003.columns = ['좌표', 'CD_03003_이(미)용원']
build_03003.to_csv("./LSCP/전체좌표_build_03003.csv", index = False)


print("build_10201")
build_10201=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='10201'])
build_10201.columns = ['좌표', 'CD_10201_금융업소']
build_10201.to_csv("./LSCP/전체좌표_build_10201.csv", index = False)


print("build_06299")
build_06299=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06299'])
build_06299.columns = ['좌표', 'CD_06299_기타소매시장']
build_06299.to_csv("./LSCP/전체좌표_build_06299.csv", index = False)


print("build_04101")
build_04101=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04101'])
build_04101.columns = ['좌표', 'CD_04101_테니스장']
build_04101.to_csv("./LSCP/전체좌표_build_04101.csv", index = False)


print("build_08104")
build_08104=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08104'])
build_08104.columns = ['좌표', 'CD_08104_대학교']
build_08104.to_csv("./LSCP/전체좌표_build_08104.csv", index = False)


print("build_16008")
build_16008=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16008'])
build_16008.columns = ['좌표', 'CD_16008_정비학원']
build_16008.to_csv("./LSCP/전체좌표_build_16008.csv", index = False)


print("build_08202")
build_08202=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08202'])
build_08202.columns = ['좌표', 'CD_08202_영유아보육시설']
build_08202.to_csv("./LSCP/전체좌표_build_08202.csv", index = False)


print("build_05501")
build_05501=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05501'])
build_05501.columns = ['좌표', 'CD_05501_박물관']
build_05501.to_csv("./LSCP/전체좌표_build_05501.csv", index = False)


print("build_11101")
build_11101=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='11101'])
build_11101.columns = ['좌표', 'CD_11101_호텔']
build_11101.to_csv("./LSCP/전체좌표_build_11101.csv", index = False)


print("build_04020")
build_04020=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04020'])
build_04020.columns = ['좌표', 'CD_04020_노래연습장']
build_04020.to_csv("./LSCP/전체좌표_build_04020.csv", index = False)


print("build_04203")
build_04203=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04203'])
build_04203.columns = ['좌표', 'CD_04203_사찰']
build_04203.to_csv("./LSCP/전체좌표_build_04203.csv", index = False)


print("build_16004")
build_16004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16004'])
build_16004.columns = ['좌표', 'CD_16004_검사장']
build_16004.to_csv("./LSCP/전체좌표_build_16004.csv", index = False)


print("build_16003")
build_16003=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16003'])
build_16003.columns = ['좌표', 'CD_16003_폐차장']
build_16003.to_csv("./LSCP/전체좌표_build_16003.csv", index = False)


print("build_03009")
build_03009=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03009'])
build_03009.columns = ['좌표', 'CD_03009_양수장']
build_03009.to_csv("./LSCP/전체좌표_build_03009.csv", index = False)


print("build_04105")
build_04105=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04105'])
build_04105.columns = ['좌표', 'CD_04105_당구장']
build_04105.to_csv("./LSCP/전체좌표_build_04105.csv", index = False)


print("build_08199")
build_08199=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08199'])
build_08199.columns = ['좌표', 'CD_08199_기타학교']
build_08199.to_csv("./LSCP/전체좌표_build_08199.csv", index = False)


print("build_06305")
build_06305=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06305'])
build_06305.columns = ['좌표', 'CD_06305_철도역사']
build_06305.to_csv("./LSCP/전체좌표_build_06305.csv", index = False)


print("build_08204")
build_08204=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08204'])
build_08204.columns = ['좌표', 'CD_08204_아동복지시설']
build_08204.to_csv("./LSCP/전체좌표_build_08204.csv", index = False)


print("build_03022")
build_03022=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03022'])
build_03022.columns = ['좌표', 'CD_03022_마을공동작업소']
build_03022.to_csv("./LSCP/전체좌표_build_03022.csv", index = False)


print("build_17104")
build_17104=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17104'])
build_17104.columns = ['좌표', 'CD_17104_양어시설']
build_17104.to_csv("./LSCP/전체좌표_build_17104.csv", index = False)


print("build_05102")
build_05102=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05102'])
build_05102.columns = ['좌표', 'CD_05102_성당']
build_05102.to_csv("./LSCP/전체좌표_build_05102.csv", index = False)


print("build_03013")
build_03013=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03013'])
build_03013.columns = ['좌표', 'CD_03013_세탁소']
build_03013.to_csv("./LSCP/전체좌표_build_03013.csv", index = False)


print("build_03023")
build_03023=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03023'])
build_03023.columns = ['좌표', 'CD_03023_마을공동구판장']
build_03023.to_csv("./LSCP/전체좌표_build_03023.csv", index = False)


print("build_20999")
build_20999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='20999'])
build_20999.columns = ['좌표', 'CD_20999_기타묘지관련시설']
build_20999.to_csv("./LSCP/전체좌표_build_20999.csv", index = False)


print("build_07108")
build_07108=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='07108'])
build_07108.columns = ['좌표', 'CD_07108']
build_07108.to_csv("./LSCP/전체좌표_build_07108.csv", index = False)


print("build_21006")
build_21006=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='21006'])
build_21006.columns = ['좌표', 'CD_21006_관광지시설']
build_21006.to_csv("./LSCP/전체좌표_build_21006.csv", index = False)


print("build_05108")
build_05108=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05108'])
build_05108.columns = ['좌표', 'CD_05108_사당']
build_05108.to_csv("./LSCP/전체좌표_build_05108.csv", index = False)


print("build_08001")
build_08001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08001'])
build_08001.columns = ['좌표', 'CD_08001_교육(연수)원']
build_08001.to_csv("./LSCP/전체좌표_build_08001.csv", index = False)


print("build_08299")
build_08299=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08299'])
build_08299.columns = ['좌표', 'CD_08299_기타아동관련시설']
build_08299.to_csv("./LSCP/전체좌표_build_08299.csv", index = False)


print("build_04303")
build_04303=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04303'])
build_04303.columns = ['좌표', 'CD_04303_연예장']
build_04303.to_csv("./LSCP/전체좌표_build_04303.csv", index = False)


print("build_06310")
build_06310=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06310'])
build_06310.columns = ['좌표', 'CD_06310_복합유통.제공업소']
build_06310.to_csv("./LSCP/전체좌표_build_06310.csv", index = False)


print("build_05106")
build_05106=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05106'])
build_05106.columns = ['좌표', 'CD_05106_수녀원']
build_05106.to_csv("./LSCP/전체좌표_build_05106.csv", index = False)


print("build_03102")
build_03102=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03102'])
build_03102.columns = ['좌표', 'CD_03102_경찰서']
build_03102.to_csv("./LSCP/전체좌표_build_03102.csv", index = False)


print("build_06205")
build_06205=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06205'])
build_06205.columns = ['좌표', 'CD_06205_대규모소매점']
build_06205.to_csv("./LSCP/전체좌표_build_06205.csv", index = False)


print("build_09001")
build_09001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='09001'])
build_09001.columns = ['좌표', 'CD_09001_체육관']
build_09001.to_csv("./LSCP/전체좌표_build_09001.csv", index = False)


print("build_17006")
build_17006=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17006'])
build_17006.columns = ['좌표', 'CD_17006_종묘배양시설']
build_17006.to_csv("./LSCP/전체좌표_build_17006.csv", index = False)


print("build_05105")
build_05105=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05105'])
build_05105.columns = ['좌표', 'CD_05105_수도원']
build_05105.to_csv("./LSCP/전체좌표_build_05105.csv", index = False)


print("build_05299")
build_05299=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05299'])
build_05299.columns = ['좌표', 'CD_05299_기타공연장']
build_05299.to_csv("./LSCP/전체좌표_build_05299.csv", index = False)


print("build_05404")
build_05404=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05404'])
build_05404.columns = ['좌표', 'CD_05404_운동장']
build_05404.to_csv("./LSCP/전체좌표_build_05404.csv", index = False)


print("build_04016")
build_04016=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04016'])
build_04016.columns = ['좌표', 'CD_04016_단란주점']
build_04016.to_csv("./LSCP/전체좌표_build_04016.csv", index = False)


print("build_18001")
build_18001=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='18001'])
build_18001.columns = ['좌표', 'CD_18001_분뇨처리시설']
build_18001.to_csv("./LSCP/전체좌표_build_18001.csv", index = False)


print("build_03011")
build_03011=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03011'])
build_03011.columns = ['좌표', 'CD_03011_대피소']
build_03011.to_csv("./LSCP/전체좌표_build_03011.csv", index = False)


print("build_05104")
build_05104=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05104'])
build_05104.columns = ['좌표', 'CD_05104_기도원']
build_05104.to_csv("./LSCP/전체좌표_build_05104.csv", index = False)


print("build_03108")
build_03108=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03108'])
build_03108.columns = ['좌표', 'CD_03108_보건소']
build_03108.to_csv("./LSCP/전체좌표_build_03108.csv", index = False)


print("build_07199")
build_07199=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='07199'])
build_07199.columns = ['좌표', 'CD_07199']
build_07199.to_csv("./LSCP/전체좌표_build_07199.csv", index = False)


print("build_06202")
build_06202=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06202'])
build_06202.columns = ['좌표', 'CD_06202_백화점']
build_06202.to_csv("./LSCP/전체좌표_build_06202.csv", index = False)


print("build_03016")
build_03016=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03016'])
build_03016.columns = ['좌표', 'CD_03016_침술원']
build_03016.to_csv("./LSCP/전체좌표_build_03016.csv", index = False)


print("build_04202")
build_04202=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04202'])
build_04202.columns = ['좌표', 'CD_04202_성당']
build_04202.to_csv("./LSCP/전체좌표_build_04202.csv", index = False)


print("build_19004")
build_19004=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='19004'])
build_19004.columns = ['좌표', 'CD_19004_발전소']
build_19004.to_csv("./LSCP/전체좌표_build_19004.csv", index = False)


print("build_03014")
build_03014=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03014'])
build_03014.columns = ['좌표', 'CD_03014_치과의원']
build_03014.to_csv("./LSCP/전체좌표_build_03014.csv", index = False)


print("build_12999")
build_12999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='12999'])
build_12999.columns = ['좌표', 'CD_12999_기타위락시설']
build_12999.to_csv("./LSCP/전체좌표_build_12999.csv", index = False)


print("build_05301")
build_05301=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05301'])
build_05301.columns = ['좌표', 'CD_05301_예식장']
build_05301.to_csv("./LSCP/전체좌표_build_05301.csv", index = False)


print("build_13200")
build_13200=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='13200'])
build_13200.columns = ['좌표', 'CD_13200_공해공장']
build_13200.to_csv("./LSCP/전체좌표_build_13200.csv", index = False)


print("build_19006")
build_19006=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='19006'])
build_19006.columns = ['좌표', 'CD_19006_전신전화국']
build_19006.to_csv("./LSCP/전체좌표_build_19006.csv", index = False)


print("build_04107")
build_04107=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04107'])
build_04107.columns = ['좌표', 'CD_04107_골프연습장']
build_04107.to_csv("./LSCP/전체좌표_build_04107.csv", index = False)


print("build_07101")
build_07101=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='07101'])
build_07101.columns = ['좌표', 'CD_07101_종합병원']
build_07101.to_csv("./LSCP/전체좌표_build_07101.csv", index = False)


print("build_06204")
build_06204=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06204'])
build_06204.columns = ['좌표', 'CD_06204_대형점']
build_06204.to_csv("./LSCP/전체좌표_build_06204.csv", index = False)


print("build_04022")
build_04022=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04022'])
build_04022.columns = ['좌표', 'CD_04022_멀티미디어문화콘텐츠 설비제공업소']
build_04022.to_csv("./LSCP/전체좌표_build_04022.csv", index = False)


print("build_07102")
build_07102=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='07102'])
build_07102.columns = ['좌표', 'CD_07102_산부인과병원']
build_07102.to_csv("./LSCP/전체좌표_build_07102.csv", index = False)


print("build_06303")
build_06303=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06303'])
build_06303.columns = ['좌표', 'CD_06303_여객자동차터미널']
build_06303.to_csv("./LSCP/전체좌표_build_06303.csv", index = False)


print("build_15006")
build_15006=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='15006'])
build_15006.columns = ['좌표', 'CD_15006_액화가스판매소']
build_15006.to_csv("./LSCP/전체좌표_build_15006.csv", index = False)


print("build_08601")
build_08601=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08601'])
build_08601.columns = ['좌표', 'CD_08601_청소년수련원(관)']
build_08601.to_csv("./LSCP/전체좌표_build_08601.csv", index = False)


print("build_15999")
build_15999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='15999'])
build_15999.columns = ['좌표', 'CD_15999_기타위험물저장처리시설']
build_15999.to_csv("./LSCP/전체좌표_build_15999.csv", index = False)


print("build_04405")
build_04405=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04405'])
build_04405.columns = ['좌표', 'CD_04405_출판사']
build_04405.to_csv("./LSCP/전체좌표_build_04405.csv", index = False)


print("build_16007")
build_16007=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='16007'])
build_16007.columns = ['좌표', 'CD_16007_운전학원']
build_16007.to_csv("./LSCP/전체좌표_build_16007.csv", index = False)


print("build_03019")
build_03019=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03019'])
build_03019.columns = ['좌표', 'CD_03019_탁구장']
build_03019.to_csv("./LSCP/전체좌표_build_03019.csv", index = False)


print("build_03017")
build_03017=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03017'])
build_03017.columns = ['좌표', 'CD_03017_접골원']
build_03017.to_csv("./LSCP/전체좌표_build_03017.csv", index = False)


print("build_03018")
build_03018=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03018'])
build_03018.columns = ['좌표', 'CD_03018_조산원']
build_03018.to_csv("./LSCP/전체좌표_build_03018.csv", index = False)


print("build_04023")
build_04023=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04023'])
build_04023.columns = ['좌표', 'CD_04023_복합유통제공업소']
build_04023.to_csv("./LSCP/전체좌표_build_04023.csv", index = False)


print("build_11299")
build_11299=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='11299'])
build_11299.columns = ['좌표', 'CD_11299_기타관광숙박시설']
build_11299.to_csv("./LSCP/전체좌표_build_11299.csv", index = False)


print("build_04008")
build_04008=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04008'])
build_04008.columns = ['좌표', 'CD_04008_삭진관']
build_04008.to_csv("./LSCP/전체좌표_build_04008.csv", index = False)


print("build_09005")
build_09005=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='09005'])
build_09005.columns = ['좌표', 'CD_09005_테니스장']
build_09005.to_csv("./LSCP/전체좌표_build_09005.csv", index = False)


print("build_03106")
build_03106=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03106'])
build_03106.columns = ['좌표', 'CD_03106_전신전화국']
build_03106.to_csv("./LSCP/전체좌표_build_03106.csv", index = False)


print("build_05303")
build_05303=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05303'])
build_05303.columns = ['좌표', 'CD_05303_공회당']
build_05303.to_csv("./LSCP/전체좌표_build_05303.csv", index = False)


print("build_05699")
build_05699=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05699'])
build_05699.columns = ['좌표', 'CD_05699_기타동.식물원']
build_05699.to_csv("./LSCP/전체좌표_build_05699.csv", index = False)


print("build_04399")
build_04399=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04399'])
build_04399.columns = ['좌표', 'CD_04399_기타공연장']
build_04399.to_csv("./LSCP/전체좌표_build_04399.csv", index = False)


print("build_05403")
build_05403=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05403'])
build_05403.columns = ['좌표', 'CD_05403_체육관']
build_05403.to_csv("./LSCP/전체좌표_build_05403.csv", index = False)


print("build_15007")
build_15007=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='15007'])
build_15007.columns = ['좌표', 'CD_15007_유독물보관저장시설']
build_15007.to_csv("./LSCP/전체좌표_build_15007.csv", index = False)


print("build_05505")
build_05505=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05505'])
build_05505.columns = ['좌표', 'CD_05505_산업전시장']
build_05505.to_csv("./LSCP/전체좌표_build_05505.csv", index = False)


print("build_04021")
build_04021=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='04021'])
build_04021.columns = ['좌표', 'CD_04021_세탁소']
build_04021.to_csv("./LSCP/전체좌표_build_04021.csv", index = False)


print("build_09011")
build_09011=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='09011'])
build_09011.columns = ['좌표', 'CD_09011_골프연습장']
build_09011.to_csv("./LSCP/전체좌표_build_09011.csv", index = False)


print("build_20002")
build_20002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='20002'])
build_20002.columns = ['좌표', 'CD_20002_납골당']
build_20002.to_csv("./LSCP/전체좌표_build_20002.csv", index = False)


print("build_06201")
build_06201=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='06201'])
build_06201.columns = ['좌표', 'CD_06201_시장']
build_06201.to_csv("./LSCP/전체좌표_build_06201.csv", index = False)


print("build_17105")
build_17105=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='17105'])
build_17105.columns = ['좌표', 'CD_17105_부화장']
build_17105.to_csv("./LSCP/전체좌표_build_17105.csv", index = False)


print("build_08699")
build_08699=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='08699'])
build_08699.columns = ['좌표', 'CD_08699_기타생활권수련시설']
build_08699.to_csv("./LSCP/전체좌표_build_08699.csv", index = False)


print("build_03109")
build_03109=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='03109'])
build_03109.columns = ['좌표', 'CD_03109_공공도서관']
build_03109.to_csv("./LSCP/전체좌표_build_03109.csv", index = False)


print("build_27999")
build_27999=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='27999'])
build_27999.columns = ['좌표', 'CD_27999']
build_27999.to_csv("./LSCP/전체좌표_build_27999.csv", index = False)


print("build_90002")
build_90002=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='90002'])
build_90002.columns = ['좌표', 'CD_90002']
build_90002.to_csv("./LSCP/전체좌표_build_90002.csv", index = False)


print("build_05502")
build_05502=geodis2(lscp_all,build_road[build_road['BDTYP_CD']=='05502'])
build_05502.columns = ['좌표', 'CD_05502_미술관']
build_05502.to_csv("./LSCP/전체좌표_build_05502.csv", index = False)



build_01003



build_02003



build_03999



build_03005



build_01001


...


build_08699



build_03109



build_27999



build_90002



build_05502


# 데이터 병합

In [ ]:
lscp_all

,좌표,위도,경도
0,"(37.57961579548528, 126.87506406643347)",37.579616,126.875064
1,"(37.57963617229605, 126.87619583846381)",37.579636,126.876196
2,"(37.57965653826528, 126.87732761163377)",37.579657,126.877328
3,"(37.58049614012486, 126.87390667102693)",37.580496,126.873907
4,"(37.5805165284361, 126.8750384555327)",37.580517,126.875038
5,"(37.58053690590551, 126.87617024117878)",37.580537,126.876170
6,"(37.58055727253304, 126.87730202796448)",37.580557,126.877302
7,"(37.58106917422839, 126.85577243099591)",37.581069,126.855772
8,"(37.58108973666921, 126.85690421079029)",37.581090,126.856904
9,"(37.581110288268384, 126.85803599173475)",37.581110,126.858036


In [ ]:
df_resident = pd.read_csv('./LSCP/전체좌표_resident.csv').drop(['Unnamed: 0'], axis=1)
df_foot = pd.read_csv('./LSCP/전체좌표_foottraffic300.csv').drop(['Unnamed: 0'], axis=1)
godo_350 = pd.read_csv('./LSCP/전체좌표_godo_300.csv').drop(['Unnamed: 0'], axis=1)
bike_100 = pd.read_csv('./LSCP/전체좌표_bike_100.csv').drop(['Unnamed: 0'], axis=1)
bike_parking = pd.read_csv('./LSCP/전체좌표_parking_350.csv').drop(['Unnamed: 0'], axis=1)
bike_cult = pd.read_csv('./LSCP/전체좌표_culture_350.csv').drop(['Unnamed: 0'], axis=1)
bike_physi = pd.read_csv('./LSCP/전체좌표_physical_350.csv').drop(['Unnamed: 0'], axis=1)
bike_fes = pd.read_csv('./LSCP/전체좌표_KINTEX_600.csv').drop(['Unnamed: 0'], axis=1)
bike_hosu = pd.read_csv('./LSCP/전체좌표_hosu_300.csv').drop(['Unnamed: 0'], axis=1)
bike_park = pd.read_csv('./LSCP/전체좌표_park_300.csv').drop(['Unnamed: 0'], axis=1)

In [ ]:
res_base =lscp_base.merge(df_resident, on ='좌표')
foot_res = res_base.merge(df_foot, on ='좌표')
bus_godo=foot_res.merge(godo_350,on='좌표')
godo_bike=bus_godo.merge(bike_100,on='좌표')
bi_parking=godo_bike.merge(bike_parking,on='좌표')
parking_cult=bi_parking.merge(bike_cult,on='좌표')
cult_physi=parking_cult.merge(bike_physi,on='좌표')
physi_fes=cult_physi.merge(bike_fes,on='좌표')
fes_hosu=physi_fes.merge(bike_hosu,on='좌표')
hosu_park=fes_hosu.merge(bike_park,on='좌표')

In [ ]:
hosu_park

,좌표,주거인구,버스_승하차수,지하철_승하차수,유동인구,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count
0,"(37.57961579548528, 126.87506406643347)",0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0
1,"(37.57963617229605, 126.87619583846381)",0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0
2,"(37.57965653826528, 126.87732761163377)",0.0,0.0,0.0,0.0,21.0,1,0,0,0,0,0,0
3,"(37.58049614012486, 126.87390667102693)",0.0,0.0,0.0,0.0,17.0,1,0,0,0,0,0,0
4,"(37.5805165284361, 126.8750384555327)",0.0,0.0,0.0,0.0,16.0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,0,0
20833,"(37.747040166676285, 126.91454518217792)",0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,0,0
20834,"(37.74706024022562, 126.9156795278339)",0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,0,0
20835,"(37.60951982027619, 126.83456514393055)",0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,0,0


In [ ]:
# 도로명주소 파일 불러오는 명령문 출력 
for x in 도로명:
  print(f'{x} = pd.read_csv("./LSCP/전체좌표_{x}.csv")')

build_01003 = pd.read_csv("./LSCP/전체좌표_build_01003.csv")
build_02003 = pd.read_csv("./LSCP/전체좌표_build_02003.csv")
build_03999 = pd.read_csv("./LSCP/전체좌표_build_03999.csv")
build_03005 = pd.read_csv("./LSCP/전체좌표_build_03005.csv")
build_01001 = pd.read_csv("./LSCP/전체좌표_build_01001.csv")
build_13100 = pd.read_csv("./LSCP/전체좌표_build_13100.csv")
build_16006 = pd.read_csv("./LSCP/전체좌표_build_16006.csv")
build_03001 = pd.read_csv("./LSCP/전체좌표_build_03001.csv")
build_17007 = pd.read_csv("./LSCP/전체좌표_build_17007.csv")
build_16010 = pd.read_csv("./LSCP/전체좌표_build_16010.csv")
build_04001 = pd.read_csv("./LSCP/전체좌표_build_04001.csv")
build_03104 = pd.read_csv("./LSCP/전체좌표_build_03104.csv")
build_14999 = pd.read_csv("./LSCP/전체좌표_build_14999.csv")
build_04005 = pd.read_csv("./LSCP/전체좌표_build_04005.csv")
build_04010 = pd.read_csv("./LSCP/전체좌표_build_04010.csv")
build_04004 = pd.read_csv("./LSCP/전체좌표_build_04004.csv")
build_14001 = pd.read_csv("./LSCP/전체좌표_build_14001.csv")
build_04999 = pd.read_csv("./LS

In [ ]:
# 도로명주소 파일 불러오는 명령문 실행 
build_01003 = pd.read_csv("./LSCP/전체좌표_build_01003.csv")
build_02003 = pd.read_csv("./LSCP/전체좌표_build_02003.csv")
build_03999 = pd.read_csv("./LSCP/전체좌표_build_03999.csv")
build_03005 = pd.read_csv("./LSCP/전체좌표_build_03005.csv")
build_01001 = pd.read_csv("./LSCP/전체좌표_build_01001.csv")
build_13100 = pd.read_csv("./LSCP/전체좌표_build_13100.csv")
build_16006 = pd.read_csv("./LSCP/전체좌표_build_16006.csv")
build_03001 = pd.read_csv("./LSCP/전체좌표_build_03001.csv")
build_17007 = pd.read_csv("./LSCP/전체좌표_build_17007.csv")
build_16010 = pd.read_csv("./LSCP/전체좌표_build_16010.csv")
build_04001 = pd.read_csv("./LSCP/전체좌표_build_04001.csv")
build_03104 = pd.read_csv("./LSCP/전체좌표_build_03104.csv")
build_14999 = pd.read_csv("./LSCP/전체좌표_build_14999.csv")
build_04005 = pd.read_csv("./LSCP/전체좌표_build_04005.csv")
build_04010 = pd.read_csv("./LSCP/전체좌표_build_04010.csv")
build_04004 = pd.read_csv("./LSCP/전체좌표_build_04004.csv")
build_14001 = pd.read_csv("./LSCP/전체좌표_build_14001.csv")
build_04999 = pd.read_csv("./LSCP/전체좌표_build_04999.csv")
build_19003 = pd.read_csv("./LSCP/전체좌표_build_19003.csv")
build_04006 = pd.read_csv("./LSCP/전체좌표_build_04006.csv")
build_02002 = pd.read_csv("./LSCP/전체좌표_build_02002.csv")
build_01002 = pd.read_csv("./LSCP/전체좌표_build_01002.csv")
build_04402 = pd.read_csv("./LSCP/전체좌표_build_04402.csv")
build_04403 = pd.read_csv("./LSCP/전체좌표_build_04403.csv")
build_08102 = pd.read_csv("./LSCP/전체좌표_build_08102.csv")
build_17999 = pd.read_csv("./LSCP/전체좌표_build_17999.csv")
build_10202 = pd.read_csv("./LSCP/전체좌표_build_10202.csv")
build_17101 = pd.read_csv("./LSCP/전체좌표_build_17101.csv")
build_17203 = pd.read_csv("./LSCP/전체좌표_build_17203.csv")
build_03020 = pd.read_csv("./LSCP/전체좌표_build_03020.csv")
build_19007 = pd.read_csv("./LSCP/전체좌표_build_19007.csv")
build_04299 = pd.read_csv("./LSCP/전체좌표_build_04299.csv")
build_08201 = pd.read_csv("./LSCP/전체좌표_build_08201.csv")
build_16001 = pd.read_csv("./LSCP/전체좌표_build_16001.csv")
build_06999 = pd.read_csv("./LSCP/전체좌표_build_06999.csv")
build_03021 = pd.read_csv("./LSCP/전체좌표_build_03021.csv")
build_04199 = pd.read_csv("./LSCP/전체좌표_build_04199.csv")
build_05107 = pd.read_csv("./LSCP/전체좌표_build_05107.csv")
build_08003 = pd.read_csv("./LSCP/전체좌표_build_08003.csv")
build_08101 = pd.read_csv("./LSCP/전체좌표_build_08101.csv")
build_15001 = pd.read_csv("./LSCP/전체좌표_build_15001.csv")
build_06203 = pd.read_csv("./LSCP/전체좌표_build_06203.csv")
build_08300 = pd.read_csv("./LSCP/전체좌표_build_08300.csv")
build_02001 = pd.read_csv("./LSCP/전체좌표_build_02001.csv")
build_04499 = pd.read_csv("./LSCP/전체좌표_build_04499.csv")
build_17299 = pd.read_csv("./LSCP/전체좌표_build_17299.csv")
build_05999 = pd.read_csv("./LSCP/전체좌표_build_05999.csv")
build_07107 = pd.read_csv("./LSCP/전체좌표_build_07107.csv")
build_16002 = pd.read_csv("./LSCP/전체좌표_build_16002.csv")
build_03199 = pd.read_csv("./LSCP/전체좌표_build_03199.csv")
build_04102 = pd.read_csv("./LSCP/전체좌표_build_04102.csv")
build_03007 = pd.read_csv("./LSCP/전체좌표_build_03007.csv")
build_03002 = pd.read_csv("./LSCP/전체좌표_build_03002.csv")
build_18003 = pd.read_csv("./LSCP/전체좌표_build_18003.csv")
build_05101 = pd.read_csv("./LSCP/전체좌표_build_05101.csv")
build_05199 = pd.read_csv("./LSCP/전체좌표_build_05199.csv")
build_19005 = pd.read_csv("./LSCP/전체좌표_build_19005.csv")
build_08999 = pd.read_csv("./LSCP/전체좌표_build_08999.csv")
build_15005 = pd.read_csv("./LSCP/전체좌표_build_15005.csv")
build_01004 = pd.read_csv("./LSCP/전체좌표_build_01004.csv")
build_04002 = pd.read_csv("./LSCP/전체좌표_build_04002.csv")
build_18002 = pd.read_csv("./LSCP/전체좌표_build_18002.csv")
build_16005 = pd.read_csv("./LSCP/전체좌표_build_16005.csv")
build_90001 = pd.read_csv("./LSCP/전체좌표_build_90001.csv")
build_05599 = pd.read_csv("./LSCP/전체좌표_build_05599.csv")
build_06301 = pd.read_csv("./LSCP/전체좌표_build_06301.csv")
build_03012 = pd.read_csv("./LSCP/전체좌표_build_03012.csv")
build_08203 = pd.read_csv("./LSCP/전체좌표_build_08203.csv")
build_04018 = pd.read_csv("./LSCP/전체좌표_build_04018.csv")
build_04007 = pd.read_csv("./LSCP/전체좌표_build_04007.csv")
build_09999 = pd.read_csv("./LSCP/전체좌표_build_09999.csv")
build_02007 = pd.read_csv("./LSCP/전체좌표_build_02007.csv")
build_11102 = pd.read_csv("./LSCP/전체좌표_build_11102.csv")
build_17204 = pd.read_csv("./LSCP/전체좌표_build_17204.csv")
build_08004 = pd.read_csv("./LSCP/전체좌표_build_08004.csv")
build_04014 = pd.read_csv("./LSCP/전체좌표_build_04014.csv")
build_11201 = pd.read_csv("./LSCP/전체좌표_build_11201.csv")
build_05499 = pd.read_csv("./LSCP/전체좌표_build_05499.csv")
build_08005 = pd.read_csv("./LSCP/전체좌표_build_08005.csv")
build_15002 = pd.read_csv("./LSCP/전체좌표_build_15002.csv")
build_04201 = pd.read_csv("./LSCP/전체좌표_build_04201.csv")
build_07999 = pd.read_csv("./LSCP/전체좌표_build_07999.csv")
build_09004 = pd.read_csv("./LSCP/전체좌표_build_09004.csv")
build_17004 = pd.read_csv("./LSCP/전체좌표_build_17004.csv")
build_17005 = pd.read_csv("./LSCP/전체좌표_build_17005.csv")
build_08800 = pd.read_csv("./LSCP/전체좌표_build_08800.csv")
build_04012 = pd.read_csv("./LSCP/전체좌표_build_04012.csv")
build_03107 = pd.read_csv("./LSCP/전체좌표_build_03107.csv")
build_02004 = pd.read_csv("./LSCP/전체좌표_build_02004.csv")
build_02006 = pd.read_csv("./LSCP/전체좌표_build_02006.csv")
build_02005 = pd.read_csv("./LSCP/전체좌표_build_02005.csv")
build_03015 = pd.read_csv("./LSCP/전체좌표_build_03015.csv")
build_04401 = pd.read_csv("./LSCP/전체좌표_build_04401.csv")
build_10102 = pd.read_csv("./LSCP/전체좌표_build_10102.csv")
build_03008 = pd.read_csv("./LSCP/전체좌표_build_03008.csv")
build_08400 = pd.read_csv("./LSCP/전체좌표_build_08400.csv")
build_10299 = pd.read_csv("./LSCP/전체좌표_build_10299.csv")
build_16009 = pd.read_csv("./LSCP/전체좌표_build_16009.csv")
build_10204 = pd.read_csv("./LSCP/전체좌표_build_10204.csv")
build_16999 = pd.read_csv("./LSCP/전체좌표_build_16999.csv")
build_10199 = pd.read_csv("./LSCP/전체좌표_build_10199.csv")
build_03010 = pd.read_csv("./LSCP/전체좌표_build_03010.csv")
build_03004 = pd.read_csv("./LSCP/전체좌표_build_03004.csv")
build_19999 = pd.read_csv("./LSCP/전체좌표_build_19999.csv")
build_08500 = pd.read_csv("./LSCP/전체좌표_build_08500.csv")
build_05103 = pd.read_csv("./LSCP/전체좌표_build_05103.csv")
build_08602 = pd.read_csv("./LSCP/전체좌표_build_08602.csv")
build_21999 = pd.read_csv("./LSCP/전체좌표_build_21999.csv")
build_07105 = pd.read_csv("./LSCP/전체좌표_build_07105.csv")
build_07302 = pd.read_csv("./LSCP/전체좌표_build_07302.csv")
build_04003 = pd.read_csv("./LSCP/전체좌표_build_04003.csv")
build_14002 = pd.read_csv("./LSCP/전체좌표_build_14002.csv")
build_03006 = pd.read_csv("./LSCP/전체좌표_build_03006.csv")
build_17003 = pd.read_csv("./LSCP/전체좌표_build_17003.csv")
build_04017 = pd.read_csv("./LSCP/전체좌표_build_04017.csv")
build_07104 = pd.read_csv("./LSCP/전체좌표_build_07104.csv")
build_03103 = pd.read_csv("./LSCP/전체좌표_build_03103.csv")
build_03101 = pd.read_csv("./LSCP/전체좌표_build_03101.csv")
build_08103 = pd.read_csv("./LSCP/전체좌표_build_08103.csv")
build_03105 = pd.read_csv("./LSCP/전체좌표_build_03105.csv")
build_03100 = pd.read_csv("./LSCP/전체좌표_build_03100.csv")
build_05109 = pd.read_csv("./LSCP/전체좌표_build_05109.csv")
build_15009 = pd.read_csv("./LSCP/전체좌표_build_15009.csv")
build_18999 = pd.read_csv("./LSCP/전체좌표_build_18999.csv")
build_20003 = pd.read_csv("./LSCP/전체좌표_build_20003.csv")
build_18004 = pd.read_csv("./LSCP/전체좌표_build_18004.csv")
build_17207 = pd.read_csv("./LSCP/전체좌표_build_17207.csv")
build_11199 = pd.read_csv("./LSCP/전체좌표_build_11199.csv")
build_04009 = pd.read_csv("./LSCP/전체좌표_build_04009.csv")
build_04505 = pd.read_csv("./LSCP/전체좌표_build_04505.csv")
build_06100 = pd.read_csv("./LSCP/전체좌표_build_06100.csv")
build_03003 = pd.read_csv("./LSCP/전체좌표_build_03003.csv")
build_10201 = pd.read_csv("./LSCP/전체좌표_build_10201.csv")
build_06299 = pd.read_csv("./LSCP/전체좌표_build_06299.csv")
build_04101 = pd.read_csv("./LSCP/전체좌표_build_04101.csv")
build_08104 = pd.read_csv("./LSCP/전체좌표_build_08104.csv")
build_16008 = pd.read_csv("./LSCP/전체좌표_build_16008.csv")
build_08202 = pd.read_csv("./LSCP/전체좌표_build_08202.csv")
build_05501 = pd.read_csv("./LSCP/전체좌표_build_05501.csv")
build_11101 = pd.read_csv("./LSCP/전체좌표_build_11101.csv")
build_04020 = pd.read_csv("./LSCP/전체좌표_build_04020.csv")
build_04203 = pd.read_csv("./LSCP/전체좌표_build_04203.csv")
build_16004 = pd.read_csv("./LSCP/전체좌표_build_16004.csv")
build_16003 = pd.read_csv("./LSCP/전체좌표_build_16003.csv")
build_03009 = pd.read_csv("./LSCP/전체좌표_build_03009.csv")
build_04105 = pd.read_csv("./LSCP/전체좌표_build_04105.csv")
build_08199 = pd.read_csv("./LSCP/전체좌표_build_08199.csv")
build_06305 = pd.read_csv("./LSCP/전체좌표_build_06305.csv")
build_08204 = pd.read_csv("./LSCP/전체좌표_build_08204.csv")
build_03022 = pd.read_csv("./LSCP/전체좌표_build_03022.csv")
build_17104 = pd.read_csv("./LSCP/전체좌표_build_17104.csv")
build_05102 = pd.read_csv("./LSCP/전체좌표_build_05102.csv")
build_03013 = pd.read_csv("./LSCP/전체좌표_build_03013.csv")
build_03023 = pd.read_csv("./LSCP/전체좌표_build_03023.csv")
build_20999 = pd.read_csv("./LSCP/전체좌표_build_20999.csv")
build_07108 = pd.read_csv("./LSCP/전체좌표_build_07108.csv")
build_21006 = pd.read_csv("./LSCP/전체좌표_build_21006.csv")
build_05108 = pd.read_csv("./LSCP/전체좌표_build_05108.csv")
build_08001 = pd.read_csv("./LSCP/전체좌표_build_08001.csv")
build_08299 = pd.read_csv("./LSCP/전체좌표_build_08299.csv")
build_04303 = pd.read_csv("./LSCP/전체좌표_build_04303.csv")
build_06310 = pd.read_csv("./LSCP/전체좌표_build_06310.csv")
build_05106 = pd.read_csv("./LSCP/전체좌표_build_05106.csv")
build_03102 = pd.read_csv("./LSCP/전체좌표_build_03102.csv")
build_06205 = pd.read_csv("./LSCP/전체좌표_build_06205.csv")
build_09001 = pd.read_csv("./LSCP/전체좌표_build_09001.csv")
build_17006 = pd.read_csv("./LSCP/전체좌표_build_17006.csv")
build_05105 = pd.read_csv("./LSCP/전체좌표_build_05105.csv")
build_05299 = pd.read_csv("./LSCP/전체좌표_build_05299.csv")
build_05404 = pd.read_csv("./LSCP/전체좌표_build_05404.csv")
build_04016 = pd.read_csv("./LSCP/전체좌표_build_04016.csv")
build_18001 = pd.read_csv("./LSCP/전체좌표_build_18001.csv")
build_03011 = pd.read_csv("./LSCP/전체좌표_build_03011.csv")
build_05104 = pd.read_csv("./LSCP/전체좌표_build_05104.csv")
build_03108 = pd.read_csv("./LSCP/전체좌표_build_03108.csv")
build_07199 = pd.read_csv("./LSCP/전체좌표_build_07199.csv")
build_06202 = pd.read_csv("./LSCP/전체좌표_build_06202.csv")
build_03016 = pd.read_csv("./LSCP/전체좌표_build_03016.csv")
build_04202 = pd.read_csv("./LSCP/전체좌표_build_04202.csv")
build_19004 = pd.read_csv("./LSCP/전체좌표_build_19004.csv")
build_03014 = pd.read_csv("./LSCP/전체좌표_build_03014.csv")
build_12999 = pd.read_csv("./LSCP/전체좌표_build_12999.csv")
build_05301 = pd.read_csv("./LSCP/전체좌표_build_05301.csv")
build_13200 = pd.read_csv("./LSCP/전체좌표_build_13200.csv")
build_19006 = pd.read_csv("./LSCP/전체좌표_build_19006.csv")
build_04107 = pd.read_csv("./LSCP/전체좌표_build_04107.csv")
build_07101 = pd.read_csv("./LSCP/전체좌표_build_07101.csv")
build_06204 = pd.read_csv("./LSCP/전체좌표_build_06204.csv")
build_04022 = pd.read_csv("./LSCP/전체좌표_build_04022.csv")
build_07102 = pd.read_csv("./LSCP/전체좌표_build_07102.csv")
build_06303 = pd.read_csv("./LSCP/전체좌표_build_06303.csv")
build_15006 = pd.read_csv("./LSCP/전체좌표_build_15006.csv")
build_08601 = pd.read_csv("./LSCP/전체좌표_build_08601.csv")
build_15999 = pd.read_csv("./LSCP/전체좌표_build_15999.csv")
build_04405 = pd.read_csv("./LSCP/전체좌표_build_04405.csv")
build_16007 = pd.read_csv("./LSCP/전체좌표_build_16007.csv")
build_03019 = pd.read_csv("./LSCP/전체좌표_build_03019.csv")
build_03017 = pd.read_csv("./LSCP/전체좌표_build_03017.csv")
build_03018 = pd.read_csv("./LSCP/전체좌표_build_03018.csv")
build_04023 = pd.read_csv("./LSCP/전체좌표_build_04023.csv")
build_11299 = pd.read_csv("./LSCP/전체좌표_build_11299.csv")
build_04008 = pd.read_csv("./LSCP/전체좌표_build_04008.csv")
build_09005 = pd.read_csv("./LSCP/전체좌표_build_09005.csv")
build_03106 = pd.read_csv("./LSCP/전체좌표_build_03106.csv")
build_05303 = pd.read_csv("./LSCP/전체좌표_build_05303.csv")
build_05699 = pd.read_csv("./LSCP/전체좌표_build_05699.csv")
build_04399 = pd.read_csv("./LSCP/전체좌표_build_04399.csv")
build_05403 = pd.read_csv("./LSCP/전체좌표_build_05403.csv")
build_15007 = pd.read_csv("./LSCP/전체좌표_build_15007.csv")
build_05505 = pd.read_csv("./LSCP/전체좌표_build_05505.csv")
build_04021 = pd.read_csv("./LSCP/전체좌표_build_04021.csv")
build_09011 = pd.read_csv("./LSCP/전체좌표_build_09011.csv")
build_20002 = pd.read_csv("./LSCP/전체좌표_build_20002.csv")
build_06201 = pd.read_csv("./LSCP/전체좌표_build_06201.csv")
build_17105 = pd.read_csv("./LSCP/전체좌표_build_17105.csv")
build_08699 = pd.read_csv("./LSCP/전체좌표_build_08699.csv")
build_03109 = pd.read_csv("./LSCP/전체좌표_build_03109.csv")
build_27999 = pd.read_csv("./LSCP/전체좌표_build_27999.csv")
build_90002 = pd.read_csv("./LSCP/전체좌표_build_90002.csv")
build_05502 = pd.read_csv("./LSCP/전체좌표_build_05502.csv")

In [ ]:
# 코드정의서 - 건물용도코드 불러오기 
address_code = address_code[581:].reset_index().drop('index', axis = 1)
address_code.columns = ['code', 'code_name']
address_code

,code,code_name


In [ ]:
# # 도로명 주소 컬럼명 바꾸기 
# fail = []
# for x in 도로명:
#   if x[6:] not in address_code['code'].values:
#     print(f"{x}.columns = ['좌표', 'CD_{x[6:]}']") 

#   for code, code_name in zip(address_code['code'], address_code['code_name']):
#     if x[6:] == code:
#       print(f"{x}.columns = ['좌표', 'CD_{x[6:]}_{code_name}']")      

In [ ]:
build_01003.columns = ['좌표', 'CD_01003_다가구주택']
build_02003.columns = ['좌표', 'CD_02003_다세대주택']
build_03999.columns = ['좌표', 'CD_03999_기타제1종근생']
build_03005.columns = ['좌표', 'CD_03005_의원']
build_01001.columns = ['좌표', 'CD_01001_단독주택']
build_13100.columns = ['좌표', 'CD_13100_일반공장']
build_16006.columns = ['좌표', 'CD_16006_정비공장']
build_03001.columns = ['좌표', 'CD_03001_소매점']
build_17007.columns = ['좌표', 'CD_17007_온실']
build_16010.columns = ['좌표', 'CD_16010_주기장']
build_04001.columns = ['좌표', 'CD_04001_일반음식점']
build_03104.columns = ['좌표', 'CD_03104_소방서']
build_14999.columns = ['좌표', 'CD_14999_기타창고시설']
build_04005.columns = ['좌표', 'CD_04005_제조업소']
build_04010.columns = ['좌표', 'CD_04010_학원']
build_04004.columns = ['좌표', 'CD_04004_서점']
build_14001.columns = ['좌표', 'CD_14001_창고']
build_04999.columns = ['좌표', 'CD_04999_기타제2종근생']
build_19003.columns = ['좌표', 'CD_19003_군사시설']
build_04006.columns = ['좌표', 'CD_04006_수리점']
build_02002.columns = ['좌표', 'CD_02002_연립주택']
build_01002.columns = ['좌표', 'CD_01002_다중주택']
build_04402.columns = ['좌표', 'CD_04402_사무소']
build_04403.columns = ['좌표', 'CD_04403_부동산중개업소']
build_08102.columns = ['좌표', 'CD_08102_중학교']
build_17999.columns = ['좌표', 'CD_17999_기타동식물관련시설']
build_10202.columns = ['좌표', 'CD_10202_오피스텔']
build_17101.columns = ['좌표', 'CD_17101_축사']
build_17203.columns = ['좌표', 'CD_17203_관리사']
build_03020.columns = ['좌표', 'CD_03020_체육도장']
build_19007.columns = ['좌표', 'CD_19007_촬영소']
build_04299.columns = ['좌표', 'CD_04299_기타종교집회장']
build_08201.columns = ['좌표', 'CD_08201_유치원']
build_16001.columns = ['좌표', 'CD_16001_주차장']
build_06999.columns = ['좌표', 'CD_06999_기타판매및영업시설']
build_03021.columns = ['좌표', 'CD_03021_마을공회당']
build_04199.columns = ['좌표', 'CD_04199_기타운동시설']
build_05107.columns = ['좌표', 'CD_05107_제실']
build_08003.columns = ['좌표', 'CD_08003_학원']
build_08003.columns = ['좌표', 'CD_08003_어린이집']
build_08101.columns = ['좌표', 'CD_08101_초등학교']
build_15001.columns = ['좌표', 'CD_15001_주유소']
build_06203.columns = ['좌표', 'CD_06203_대형판매점']
build_08300.columns = ['좌표', 'CD_08300_노인복지시설']
build_02001.columns = ['좌표', 'CD_02001_아파트']
build_04499.columns = ['좌표', 'CD_04499_기타사무소']
build_17299.columns = ['좌표', 'CD_17299_기타가축시설']
build_05999.columns = ['좌표', 'CD_05999_기타문화및집회시설']
build_07107.columns = ['좌표', 'CD_07107_병원']
build_16002.columns = ['좌표', 'CD_16002_세차장']
build_03199.columns = ['좌표', 'CD_03199_기타공공시설']
build_04102.columns = ['좌표', 'CD_04102_체력단련장']
build_03007.columns = ['좌표', 'CD_03007_마을공동시설']
build_03002.columns = ['좌표', 'CD_03002_휴게음식점']
build_18003.columns = ['좌표', 'CD_18003_폐기물재활용시설']
build_05101.columns = ['좌표', 'CD_05101_교회']
build_05199.columns = ['좌표', 'CD_05199_기타종교집회장']
build_19005.columns = ['좌표', 'CD_19005_방송국']
build_08999.columns = ['좌표', 'CD_08999_기타교육연구및복지시설']
build_15005.columns = ['좌표', 'CD_15005_액화가스취급소']
build_01004.columns = ['좌표', 'CD_01004_공관']
build_04002.columns = ['좌표', 'CD_04002_휴게음식점']
build_18002.columns = ['좌표', 'CD_18002_폐기물처리시설']
build_16005.columns = ['좌표', 'CD_16005_매매장']
build_90001.columns = ['좌표', 'CD_90001']
build_05599.columns = ['좌표', 'CD_05599_기타전시장']
build_06301.columns = ['좌표', 'CD_06301_상점']
build_03012.columns = ['좌표', 'CD_03012_공중화장실']
build_08203.columns = ['좌표', 'CD_08203']
build_04018.columns = ['좌표', 'CD_04018_자동차영업소']
build_04007.columns = ['좌표', 'CD_04007_게임제공업소']
build_09999.columns = ['좌표', 'CD_09999_기타운동시설']
build_02007.columns = ['좌표', 'CD_02007_기숙사']
build_11102.columns = ['좌표', 'CD_11102_여관']
build_17204.columns = ['좌표', 'CD_17204_가축용창고']
build_08004.columns = ['좌표', 'CD_08004_연구소']
build_04014.columns = ['좌표', 'CD_04014_독서실']
build_11201.columns = ['좌표', 'CD_11201_관광호텔']
build_05499.columns = ['좌표', 'CD_05499_기타관람장']
build_08005.columns = ['좌표', 'CD_08005_도서관']
build_15002.columns = ['좌표', 'CD_15002_액화석유가스충전소']
build_04201.columns = ['좌표', 'CD_04201_교회']
build_07999.columns = ['좌표', 'CD_07999_기타의료시설']
build_09004.columns = ['좌표', 'CD_09004_체육도장']
build_17004.columns = ['좌표', 'CD_17004_도계장']
build_17005.columns = ['좌표', 'CD_17005_버섯재배사']
build_08800.columns = ['좌표', 'CD_08800']
build_04012.columns = ['좌표', 'CD_04012_동물병원']
build_03107.columns = ['좌표', 'CD_03107_방송국']
build_02004.columns = ['좌표', 'CD_02004_생활편익시설']
build_02006.columns = ['좌표', 'CD_02006_복리시설']
build_02005.columns = ['좌표', 'CD_02005_부대시설']
build_03015.columns = ['좌표', 'CD_03015_한의원']
build_04401.columns = ['좌표', 'CD_04401_금융업소']
build_10102.columns = ['좌표', 'CD_10102_자치단체청사']
build_03008.columns = ['좌표', 'CD_03008_변전소']
build_08400.columns = ['좌표', 'CD_08400_사회복지시설']
build_10299.columns = ['좌표', 'CD_10299_기타일반업무시설']
build_16009.columns = ['좌표', 'CD_16009_차고']
build_10204.columns = ['좌표', 'CD_10204_사무소']
build_16999.columns = ['좌표', 'CD_16999_기타자동차관련시설']
build_10199.columns = ['좌표', 'CD_10199_기타공공업무시설']
build_03010.columns = ['좌표', 'CD_03010_정수장']
build_03004.columns = ['좌표', 'CD_03004_일반목욕장']
build_19999.columns = ['좌표', 'CD_19999_기타공공용시설']
build_08500.columns = ['좌표', 'CD_08500_근로복지시설']
build_05103.columns = ['좌표', 'CD_05103_사찰']
build_08602.columns = ['좌표', 'CD_08602_유스호스텔']
build_21999.columns = ['좌표', 'CD_21999_기타관광휴게시설']
build_07105.columns = ['좌표', 'CD_07105_정신병원']
build_07302.columns = ['좌표', 'CD_07302_마약진료소']
build_04003.columns = ['좌표', 'CD_04003_기원']
build_14002.columns = ['좌표', 'CD_14002_하역장']
build_03006.columns = ['좌표', 'CD_03006_체육장']
build_17003.columns = ['좌표', 'CD_17003_도축장']
build_04017.columns = ['좌표', 'CD_04017_의약품도매점']
build_07104.columns = ['좌표', 'CD_07104_한방병원']
build_03103.columns = ['좌표', 'CD_03103_파출소']
build_03101.columns = ['좌표', 'CD_03101_동사무소']
build_08103.columns = ['좌표', 'CD_08103_고등학교']
build_03105.columns = ['좌표', 'CD_03105_우체국']
build_03100.columns = ['좌표', 'CD_03100_공공시설']
build_05109.columns = ['좌표', 'CD_05109_납골당']
build_15009.columns = ['좌표', 'CD_15009_석유판매소']
build_18999.columns = ['좌표', 'CD_18999_기타분뇨쓰레기처리시설']
build_20003.columns = ['좌표', 'CD_20003_묘지에 부수되는 건축물']
build_18004.columns = ['좌표', 'CD_18004_고물상']
build_17207.columns = ['좌표', 'CD_17207_실험동물사육시설']
build_11199.columns = ['좌표', 'CD_11199_기타일반숙박시설']
build_04009.columns = ['좌표', 'CD_04009_표구점']
build_04505.columns = ['좌표', 'CD_04505']
build_06100.columns = ['좌표', 'CD_06100_도매시장']
build_03003.columns = ['좌표', 'CD_03003_이(미)용원']
build_10201.columns = ['좌표', 'CD_10201_금융업소']
build_06299.columns = ['좌표', 'CD_06299_기타소매시장']
build_04101.columns = ['좌표', 'CD_04101_테니스장']
build_08104.columns = ['좌표', 'CD_08104_대학교']
build_16008.columns = ['좌표', 'CD_16008_정비학원']
build_08202.columns = ['좌표', 'CD_08202_영유아보육시설']
build_05501.columns = ['좌표', 'CD_05501_박물관']
build_11101.columns = ['좌표', 'CD_11101_호텔']
build_04020.columns = ['좌표', 'CD_04020_노래연습장']
build_04203.columns = ['좌표', 'CD_04203_사찰']
build_16004.columns = ['좌표', 'CD_16004_검사장']
build_16003.columns = ['좌표', 'CD_16003_폐차장']
build_03009.columns = ['좌표', 'CD_03009_양수장']
build_04105.columns = ['좌표', 'CD_04105_당구장']
build_08199.columns = ['좌표', 'CD_08199_기타학교']
build_06305.columns = ['좌표', 'CD_06305_철도역사']
build_08204.columns = ['좌표', 'CD_08204_아동복지시설']
build_03022.columns = ['좌표', 'CD_03022_마을공동작업소']
build_17104.columns = ['좌표', 'CD_17104_양어시설']
build_05102.columns = ['좌표', 'CD_05102_성당']
build_03013.columns = ['좌표', 'CD_03013_세탁소']
build_03023.columns = ['좌표', 'CD_03023_마을공동구판장']
build_20999.columns = ['좌표', 'CD_20999_기타묘지관련시설']
build_07108.columns = ['좌표', 'CD_07108']
build_21006.columns = ['좌표', 'CD_21006_관광지시설']
build_05108.columns = ['좌표', 'CD_05108_사당']
build_08001.columns = ['좌표', 'CD_08001_교육(연수)원']
build_08299.columns = ['좌표', 'CD_08299_기타아동관련시설']
build_04303.columns = ['좌표', 'CD_04303_연예장']
build_06310.columns = ['좌표', 'CD_06310_복합유통.제공업소']
build_05106.columns = ['좌표', 'CD_05106_수녀원']
build_03102.columns = ['좌표', 'CD_03102_경찰서']
build_06205.columns = ['좌표', 'CD_06205_대규모소매점']
build_09001.columns = ['좌표', 'CD_09001_체육관']
build_17006.columns = ['좌표', 'CD_17006_종묘배양시설']
build_05105.columns = ['좌표', 'CD_05105_수도원']
build_05299.columns = ['좌표', 'CD_05299_기타공연장']
build_05404.columns = ['좌표', 'CD_05404_운동장']
build_04016.columns = ['좌표', 'CD_04016_단란주점']
build_18001.columns = ['좌표', 'CD_18001_분뇨처리시설']
build_03011.columns = ['좌표', 'CD_03011_대피소']
build_05104.columns = ['좌표', 'CD_05104_기도원']
build_03108.columns = ['좌표', 'CD_03108_보건소']
build_07199.columns = ['좌표', 'CD_07199']
build_06202.columns = ['좌표', 'CD_06202_백화점']
build_03016.columns = ['좌표', 'CD_03016_침술원']
build_04202.columns = ['좌표', 'CD_04202_성당']
build_19004.columns = ['좌표', 'CD_19004_발전소']
build_03014.columns = ['좌표', 'CD_03014_치과의원']
build_12999.columns = ['좌표', 'CD_12999_기타위락시설']
build_05301.columns = ['좌표', 'CD_05301_예식장']
build_13200.columns = ['좌표', 'CD_13200_공해공장']
build_19006.columns = ['좌표', 'CD_19006_전신전화국']
build_04107.columns = ['좌표', 'CD_04107_골프연습장']
build_07101.columns = ['좌표', 'CD_07101_종합병원']
build_06204.columns = ['좌표', 'CD_06204_대형점']
build_04022.columns = ['좌표', 'CD_04022_멀티미디어문화콘텐츠 설비제공업소']
build_07102.columns = ['좌표', 'CD_07102_산부인과병원']
build_06303.columns = ['좌표', 'CD_06303_여객자동차터미널']
build_15006.columns = ['좌표', 'CD_15006_액화가스판매소']
build_08601.columns = ['좌표', 'CD_08601_청소년수련원(관)']
build_15999.columns = ['좌표', 'CD_15999_기타위험물저장처리시설']
build_04405.columns = ['좌표', 'CD_04405_출판사']
build_16007.columns = ['좌표', 'CD_16007_운전학원']
build_03019.columns = ['좌표', 'CD_03019_탁구장']
build_03017.columns = ['좌표', 'CD_03017_접골원']
build_03018.columns = ['좌표', 'CD_03018_조산원']
build_04023.columns = ['좌표', 'CD_04023_복합유통제공업소']
build_11299.columns = ['좌표', 'CD_11299_기타관광숙박시설']
build_04008.columns = ['좌표', 'CD_04008_삭진관']
build_09005.columns = ['좌표', 'CD_09005_테니스장']
build_03106.columns = ['좌표', 'CD_03106_전신전화국']
build_05303.columns = ['좌표', 'CD_05303_공회당']
build_05699.columns = ['좌표', 'CD_05699_기타동.식물원']
build_04399.columns = ['좌표', 'CD_04399_기타공연장']
build_05403.columns = ['좌표', 'CD_05403_체육관']
build_15007.columns = ['좌표', 'CD_15007_유독물보관저장시설']
build_05505.columns = ['좌표', 'CD_05505_산업전시장']
build_04021.columns = ['좌표', 'CD_04021_세탁소']
build_09011.columns = ['좌표', 'CD_09011_골프연습장']
build_20002.columns = ['좌표', 'CD_20002_납골당']
build_06201.columns = ['좌표', 'CD_06201_시장']
build_17105.columns = ['좌표', 'CD_17105_부화장']
build_08699.columns = ['좌표', 'CD_08699_기타생활권수련시설']
build_03109.columns = ['좌표', 'CD_03109_공공도서관']
build_27999.columns = ['좌표', 'CD_27999']
build_90002.columns = ['좌표', 'CD_90002']
build_05502.columns = ['좌표', 'CD_05502_미술관']

In [ ]:
build_1=lscp_base.merge(build_01003,on='좌표')
build_2=build_1.merge(build_02003,on='좌표')
build_3=build_2.merge(build_03999,on='좌표')
build_4=build_3.merge(build_03005,on='좌표')
build_5=build_4.merge(build_01001,on='좌표')
build_6=build_5.merge(build_13100,on='좌표')
build_7=build_6.merge(build_16006,on='좌표')
build_8=build_7.merge(build_03001,on='좌표')
build_9=build_8.merge(build_17007,on='좌표')
build_10=build_9.merge(build_16010,on='좌표')
build_11=build_10.merge(build_04001,on='좌표')
build_12=build_11.merge(build_03104,on='좌표')
build_13=build_12.merge(build_14999,on='좌표')
build_14=build_13.merge(build_04005,on='좌표')
build_15=build_14.merge(build_04010,on='좌표')
build_16=build_15.merge(build_04004,on='좌표')
build_17=build_16.merge(build_14001,on='좌표')
build_18=build_17.merge(build_04999,on='좌표')
build_19=build_18.merge(build_19003,on='좌표')
build_20=build_19.merge(build_04006,on='좌표')
build_21=build_20.merge(build_02002,on='좌표')
build_22=build_21.merge(build_01002,on='좌표')
build_23=build_22.merge(build_04402,on='좌표')
build_24=build_23.merge(build_04403,on='좌표')
build_25=build_24.merge(build_08102,on='좌표')
build_26=build_25.merge(build_17999,on='좌표')
build_27=build_26.merge(build_10202,on='좌표')
build_28=build_27.merge(build_17101,on='좌표')
build_29=build_28.merge(build_17203,on='좌표')
build_30=build_29.merge(build_03020,on='좌표')
build_31=build_30.merge(build_19007,on='좌표')
build_32=build_31.merge(build_04299,on='좌표')
build_33=build_32.merge(build_08201,on='좌표')
build_34=build_33.merge(build_16001,on='좌표')
build_35=build_34.merge(build_06999,on='좌표')
build_36=build_35.merge(build_03021,on='좌표')
build_37=build_36.merge(build_04199,on='좌표')
build_38=build_37.merge(build_05107,on='좌표')
build_39=build_38.merge(build_08003,on='좌표')
build_40=build_39.merge(build_08101,on='좌표')
build_41=build_40.merge(build_15001,on='좌표')
build_42=build_41.merge(build_06203,on='좌표')
build_43=build_42.merge(build_08300,on='좌표')
build_44=build_43.merge(build_02001,on='좌표')
build_45=build_44.merge(build_04499,on='좌표')
build_46=build_45.merge(build_17299,on='좌표')
build_47=build_46.merge(build_05999,on='좌표')
build_48=build_47.merge(build_07107,on='좌표')
build_49=build_48.merge(build_16002,on='좌표')
build_50=build_49.merge(build_03199,on='좌표')
build_51=build_50.merge(build_04102,on='좌표')
build_52=build_51.merge(build_03007,on='좌표')
build_53=build_52.merge(build_03002,on='좌표')
build_54=build_53.merge(build_18003,on='좌표')
build_55=build_54.merge(build_05101,on='좌표')
build_56=build_55.merge(build_05199,on='좌표')
build_57=build_56.merge(build_19005,on='좌표')
build_58=build_57.merge(build_08999,on='좌표')
build_59=build_58.merge(build_15005,on='좌표')
build_60=build_59.merge(build_01004,on='좌표')
build_61=build_60.merge(build_04002,on='좌표')
build_62=build_61.merge(build_18002,on='좌표')
build_63=build_62.merge(build_16005,on='좌표')
build_64=build_63.merge(build_90001,on='좌표')
build_65=build_64.merge(build_05599,on='좌표')
build_66=build_65.merge(build_06301,on='좌표')
build_67=build_66.merge(build_03012,on='좌표')
build_68=build_67.merge(build_08203,on='좌표')
build_69=build_68.merge(build_04018,on='좌표')
build_70=build_69.merge(build_04007,on='좌표')
build_71=build_70.merge(build_09999,on='좌표')
build_72=build_71.merge(build_02007,on='좌표')
build_73=build_72.merge(build_11102,on='좌표')
build_74=build_73.merge(build_17204,on='좌표')
build_75=build_74.merge(build_08004,on='좌표')
build_76=build_75.merge(build_04014,on='좌표')
build_77=build_76.merge(build_11201,on='좌표')
build_78=build_77.merge(build_05499,on='좌표')
build_79=build_78.merge(build_08005,on='좌표')
build_80=build_79.merge(build_15002,on='좌표')
build_81=build_80.merge(build_04201,on='좌표')
build_82=build_81.merge(build_07999,on='좌표')
build_83=build_82.merge(build_09004,on='좌표')
build_84=build_83.merge(build_17004,on='좌표')
build_85=build_84.merge(build_17005,on='좌표')
build_86=build_85.merge(build_08800,on='좌표')
build_87=build_86.merge(build_04012,on='좌표')
build_88=build_87.merge(build_03107,on='좌표')
build_89=build_88.merge(build_02004,on='좌표')
build_90=build_89.merge(build_02006,on='좌표')
build_91=build_90.merge(build_02005,on='좌표')
build_92=build_91.merge(build_03015,on='좌표')
build_93=build_92.merge(build_04401,on='좌표')
build_94=build_93.merge(build_10102,on='좌표')
build_95=build_94.merge(build_03008,on='좌표')
build_96=build_95.merge(build_08400,on='좌표')
build_97=build_96.merge(build_10299,on='좌표')
build_98=build_97.merge(build_16009,on='좌표')
build_99=build_98.merge(build_10204,on='좌표')
build_100=build_99.merge(build_16999,on='좌표')
build_101=build_100.merge(build_10199,on='좌표')
build_102=build_101.merge(build_03010,on='좌표')
build_103=build_102.merge(build_03004,on='좌표')
build_104=build_103.merge(build_19999,on='좌표')
build_105=build_104.merge(build_08500,on='좌표')
build_106=build_105.merge(build_05103,on='좌표')
build_107=build_106.merge(build_08602,on='좌표')
build_108=build_107.merge(build_21999,on='좌표')
build_109=build_108.merge(build_07105,on='좌표')
build_110=build_109.merge(build_07302,on='좌표')
build_111=build_110.merge(build_04003,on='좌표')
build_112=build_111.merge(build_14002,on='좌표')
build_113=build_112.merge(build_03006,on='좌표')
build_114=build_113.merge(build_17003,on='좌표')
build_115=build_114.merge(build_04017,on='좌표')
build_116=build_115.merge(build_07104,on='좌표')
build_117=build_116.merge(build_03103,on='좌표')
build_118=build_117.merge(build_03101,on='좌표')
build_119=build_118.merge(build_08103,on='좌표')
build_120=build_119.merge(build_03105,on='좌표')
build_121=build_120.merge(build_03100,on='좌표')
build_122=build_121.merge(build_05109,on='좌표')
build_123=build_122.merge(build_15009,on='좌표')
build_124=build_123.merge(build_18999,on='좌표')
build_125=build_124.merge(build_20003,on='좌표')
build_126=build_125.merge(build_18004,on='좌표')
build_127=build_126.merge(build_17207,on='좌표')
build_128=build_127.merge(build_11199,on='좌표')
build_129=build_128.merge(build_04009,on='좌표')
build_130=build_129.merge(build_04505,on='좌표')
build_131=build_130.merge(build_06100,on='좌표')
build_132=build_131.merge(build_03003,on='좌표')
build_133=build_132.merge(build_10201,on='좌표')
build_134=build_133.merge(build_06299,on='좌표')
build_135=build_134.merge(build_04101,on='좌표')
build_136=build_135.merge(build_08104,on='좌표')
build_137=build_136.merge(build_16008,on='좌표')
build_138=build_137.merge(build_08202,on='좌표')
build_139=build_138.merge(build_05501,on='좌표')
build_140=build_139.merge(build_11101,on='좌표')
build_141=build_140.merge(build_04020,on='좌표')
build_142=build_141.merge(build_04203,on='좌표')
build_143=build_142.merge(build_16004,on='좌표')
build_144=build_143.merge(build_16003,on='좌표')
build_145=build_144.merge(build_03009,on='좌표')
build_146=build_145.merge(build_04105,on='좌표')
build_147=build_146.merge(build_08199,on='좌표')
build_148=build_147.merge(build_06305,on='좌표')
build_149=build_148.merge(build_08204,on='좌표')
build_150=build_149.merge(build_03022,on='좌표')
build_151=build_150.merge(build_17104,on='좌표')
build_152=build_151.merge(build_05102,on='좌표')
build_153=build_152.merge(build_03013,on='좌표')
build_154=build_153.merge(build_03023,on='좌표')
build_155=build_154.merge(build_20999,on='좌표')
build_156=build_155.merge(build_07108,on='좌표')
build_157=build_156.merge(build_21006,on='좌표')
build_158=build_157.merge(build_05108,on='좌표')
build_159=build_158.merge(build_08001,on='좌표')
build_160=build_159.merge(build_08299,on='좌표')
build_161=build_160.merge(build_04303,on='좌표')
build_162=build_161.merge(build_06310,on='좌표')
build_163=build_162.merge(build_05106,on='좌표')
build_164=build_163.merge(build_03102,on='좌표')
build_165=build_164.merge(build_06205,on='좌표')
build_166=build_165.merge(build_09001,on='좌표')
build_167=build_166.merge(build_17006,on='좌표')
build_168=build_167.merge(build_05105,on='좌표')
build_169=build_168.merge(build_05299,on='좌표')
build_170=build_169.merge(build_05404,on='좌표')
build_171=build_170.merge(build_04016,on='좌표')
build_172=build_171.merge(build_18001,on='좌표')
build_173=build_172.merge(build_03011,on='좌표')
build_174=build_173.merge(build_05104,on='좌표')
build_175=build_174.merge(build_03108,on='좌표')
build_176=build_175.merge(build_07199,on='좌표')
build_177=build_176.merge(build_06202,on='좌표')
build_178=build_177.merge(build_03016,on='좌표')
build_179=build_178.merge(build_04202,on='좌표')
build_180=build_179.merge(build_19004,on='좌표')
build_181=build_180.merge(build_03014,on='좌표')
build_182=build_181.merge(build_12999,on='좌표')
build_183=build_182.merge(build_05301,on='좌표')
build_184=build_183.merge(build_13200,on='좌표')
build_185=build_184.merge(build_19006,on='좌표')
build_186=build_185.merge(build_04107,on='좌표')
build_187=build_186.merge(build_07101,on='좌표')
build_188=build_187.merge(build_06204,on='좌표')
build_189=build_188.merge(build_04022,on='좌표')
build_190=build_189.merge(build_07102,on='좌표')
build_191=build_190.merge(build_06303,on='좌표')
build_192=build_191.merge(build_15006,on='좌표')
build_193=build_192.merge(build_08601,on='좌표')
build_194=build_193.merge(build_15999,on='좌표')
build_195=build_194.merge(build_04405,on='좌표')
build_196=build_195.merge(build_16007,on='좌표')
build_197=build_196.merge(build_03019,on='좌표')
build_198=build_197.merge(build_03017,on='좌표')
build_199=build_198.merge(build_03018,on='좌표')
build_200=build_199.merge(build_04023,on='좌표')
build_201=build_200.merge(build_11299,on='좌표')
build_202=build_201.merge(build_04008,on='좌표')
build_203=build_202.merge(build_09005,on='좌표')
build_204=build_203.merge(build_03106,on='좌표')
build_205=build_204.merge(build_05303,on='좌표')
build_206=build_205.merge(build_05699,on='좌표')
build_207=build_206.merge(build_04399,on='좌표')
build_208=build_207.merge(build_05403,on='좌표')
build_209=build_208.merge(build_15007,on='좌표')
build_210=build_209.merge(build_05505,on='좌표')
build_211=build_210.merge(build_04021,on='좌표')
build_212=build_211.merge(build_09011,on='좌표')
build_213=build_212.merge(build_20002,on='좌표')
build_214=build_213.merge(build_06201,on='좌표')
build_215=build_214.merge(build_17105,on='좌표')
build_216=build_215.merge(build_08699,on='좌표')
build_217=build_216.merge(build_03109,on='좌표')
build_218=build_217.merge(build_27999,on='좌표')
build_219=build_218.merge(build_90002,on='좌표')
build_220=build_219.merge(build_05502,on='좌표')

In [ ]:
build_220

,좌표,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,CD_04402_사무소,CD_04403_부동산중개업소,CD_08102_중학교,CD_17999_기타동식물관련시설,CD_10202_오피스텔,CD_17101_축사,CD_17203_관리사,CD_03020_체육도장,CD_19007_촬영소,CD_04299_기타종교집회장,CD_08201_유치원,CD_16001_주차장,CD_06999_기타판매및영업시설,CD_03021_마을공회당,CD_04199_기타운동시설,CD_05107_제실,CD_08003_어린이집,...,CD_03014_치과의원,CD_12999_기타위락시설,CD_05301_예식장,CD_13200_공해공장,CD_19006_전신전화국,CD_04107_골프연습장,CD_07101_종합병원,CD_06204_대형점,CD_04022_멀티미디어문화콘텐츠 설비제공업소,CD_07102_산부인과병원,CD_06303_여객자동차터미널,CD_15006_액화가스판매소,CD_08601_청소년수련원(관),CD_15999_기타위험물저장처리시설,CD_04405_출판사,CD_16007_운전학원,CD_03019_탁구장,CD_03017_접골원,CD_03018_조산원,CD_04023_복합유통제공업소,CD_11299_기타관광숙박시설,CD_04008_삭진관,CD_09005_테니스장,CD_03106_전신전화국,CD_05303_공회당,CD_05699_기타동.식물원,CD_04399_기타공연장,CD_05403_체육관,CD_15007_유독물보관저장시설,CD_05505_산업전시장,CD_04021_세탁소,CD_09011_골프연습장,CD_20002_납골당,CD_06201_시장,CD_17105_부화장,CD_08699_기타생활권수련시설,CD_03109_공공도서관,CD_27999,CD_90002,CD_05502_미술관
0,"(37.57961579548528, 126.87506406643347)",0,0,2,0,3,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"(37.57963617229605, 126.87619583846381)",0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"(37.57965653826528, 126.87732761163377)",0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(37.58049614012486, 126.87390667102693)",0,0,3,0,9,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(37.5805165284361, 126.8750384555327)",0,0,3,0,7,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20833,"(37.747040166676285, 126.91454518217792)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20834,"(37.74706024022562, 126.9156795278339)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20835,"(37.60951982027619, 126.83456514393055)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
new = hosu_park.merge(build_220, on ='좌표')
new

,좌표,주거인구,버스_승하차수,지하철_승하차수,유동인구,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,CD_04402_사무소,CD_04403_부동산중개업소,CD_08102_중학교,CD_17999_기타동식물관련시설,CD_10202_오피스텔,...,CD_03014_치과의원,CD_12999_기타위락시설,CD_05301_예식장,CD_13200_공해공장,CD_19006_전신전화국,CD_04107_골프연습장,CD_07101_종합병원,CD_06204_대형점,CD_04022_멀티미디어문화콘텐츠 설비제공업소,CD_07102_산부인과병원,CD_06303_여객자동차터미널,CD_15006_액화가스판매소,CD_08601_청소년수련원(관),CD_15999_기타위험물저장처리시설,CD_04405_출판사,CD_16007_운전학원,CD_03019_탁구장,CD_03017_접골원,CD_03018_조산원,CD_04023_복합유통제공업소,CD_11299_기타관광숙박시설,CD_04008_삭진관,CD_09005_테니스장,CD_03106_전신전화국,CD_05303_공회당,CD_05699_기타동.식물원,CD_04399_기타공연장,CD_05403_체육관,CD_15007_유독물보관저장시설,CD_05505_산업전시장,CD_04021_세탁소,CD_09011_골프연습장,CD_20002_납골당,CD_06201_시장,CD_17105_부화장,CD_08699_기타생활권수련시설,CD_03109_공공도서관,CD_27999,CD_90002,CD_05502_미술관
0,"(37.57961579548528, 126.87506406643347)",0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"(37.57963617229605, 126.87619583846381)",0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"(37.57965653826528, 126.87732761163377)",0.0,0.0,0.0,0.0,21.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(37.58049614012486, 126.87390667102693)",0.0,0.0,0.0,0.0,17.0,1,0,0,0,0,0,0,0,0,3,0,9,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(37.5805165284361, 126.8750384555327)",0.0,0.0,0.0,0.0,16.0,1,0,0,0,0,0,0,0,0,3,0,7,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20833,"(37.747040166676285, 126.91454518217792)",0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20834,"(37.74706024022562, 126.9156795278339)",0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20835,"(37.60951982027619, 126.83456514393055)",0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
final = lscp_all.merge(new, on='좌표')
final

,좌표,위도,경도,주거인구,버스_승하차수,지하철_승하차수,유동인구,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,CD_04402_사무소,CD_04403_부동산중개업소,CD_08102_중학교,...,CD_03014_치과의원,CD_12999_기타위락시설,CD_05301_예식장,CD_13200_공해공장,CD_19006_전신전화국,CD_04107_골프연습장,CD_07101_종합병원,CD_06204_대형점,CD_04022_멀티미디어문화콘텐츠 설비제공업소,CD_07102_산부인과병원,CD_06303_여객자동차터미널,CD_15006_액화가스판매소,CD_08601_청소년수련원(관),CD_15999_기타위험물저장처리시설,CD_04405_출판사,CD_16007_운전학원,CD_03019_탁구장,CD_03017_접골원,CD_03018_조산원,CD_04023_복합유통제공업소,CD_11299_기타관광숙박시설,CD_04008_삭진관,CD_09005_테니스장,CD_03106_전신전화국,CD_05303_공회당,CD_05699_기타동.식물원,CD_04399_기타공연장,CD_05403_체육관,CD_15007_유독물보관저장시설,CD_05505_산업전시장,CD_04021_세탁소,CD_09011_골프연습장,CD_20002_납골당,CD_06201_시장,CD_17105_부화장,CD_08699_기타생활권수련시설,CD_03109_공공도서관,CD_27999,CD_90002,CD_05502_미술관
0,"(37.57961579548528, 126.87506406643347)",37.579616,126.875064,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"(37.57963617229605, 126.87619583846381)",37.579636,126.876196,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"(37.57965653826528, 126.87732761163377)",37.579657,126.877328,0.0,0.0,0.0,0.0,21.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(37.58049614012486, 126.87390667102693)",37.580496,126.873907,0.0,0.0,0.0,0.0,17.0,1,0,0,0,0,0,0,0,0,3,0,9,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(37.5805165284361, 126.8750384555327)",37.580517,126.875038,0.0,0.0,0.0,0.0,16.0,1,0,0,0,0,0,0,0,0,3,0,7,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"(37.58053690590551, 126.87617024117878)",37.580537,126.876170,0.0,0.0,0.0,0.0,14.0,1,0,0,0,0,0,0,0,0,3,0,6,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"(37.58055727253304, 126.87730202796448)",37.580557,126.877302,0.0,0.0,0.0,0.0,13.0,1,0,0,0,0,0,0,0,0,3,0,4,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"(37.58106917422839, 126.85577243099591)",37.581069,126.855772,0.0,0.0,0.0,0.0,13.0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"(37.58108973666921, 126.85690421079029)",37.581090,126.856904,0.0,0.0,0.0,0.0,16.0,1,0,0,1,0,0,0,0,0,1,0,4,0,0,2,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"(37.581110288268384, 126.85803599173475)",37.581110,126.858036,0.0,0.0,0.0,0.0,17.0,1,0,0,1,0,0,0,0,0,1,0,3,0,0,2,2,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 평균 ver
final.to_pickle('./모델학습용 데이터/Allpoint_lscp_all_300_도로명추가.pickle')

## 기존 데이터 프레임과 도시계획 LSCP 데이터 합치기 

###  도시계획_all (실재 + 계획)

In [ ]:
all_plan

,Unnamed: 0,좌표,100_all_count,110_all_count,120_all_count,200_all_count,210_all_count,220_all_count,221_all_count,250_all_count,260_all_count,290_all_count,310_all_count,320_all_count,330_all_count,390_all_count,500_all_count,520_all_count,기타 사회복지시설_all_count,초등학교_all_count,유치원_all_count,기타 체육시설_all_count,중학교_all_count,기타 도서관시설_all_count,청사(국가)_all_count,고등학교_all_count,공공체육시설_all_count,특수학교_all_count,각종학교_all_count,기타 공공직업훈련시설_all_count,대학_all_count,기타 운동장시설_all_count,지방자치단체_all_count,골프장_all_count,기타 연구시설_all_count,기타청소년수련시설_all_count,기타 문화시설_all_count,공관_all_count,박물관_all_count,기타 학교시설_all_count,종합운동장_all_count,노외주차장_all_count,공영차고지_all_count,기타 주차장시설_all_count,일반철도_all_count,기타철도시설_all_count,도시철도_all_count,여객자동차터미널_all_count
0,0,"(37.57961579548528, 126.87506406643347)",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,"(37.57963617229605, 126.87619583846381)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,"(37.57965653826528, 126.87732761163377)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,"(37.58049614012486, 126.87390667102693)",0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,"(37.5805165284361, 126.8750384555327)",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20832,20832,"(37.74702008221923, 126.91341083764837)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20833,20833,"(37.747040166676285, 126.91454518217792)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20834,20834,"(37.74706024022562, 126.9156795278339)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20835,20835,"(37.60951982027619, 126.83456514393055)",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
# 학습 시 동일할 칼럼이 필요하므로 칼럼명 동일하게 맞춰주기 
all_plan = all_plan.drop('Unnamed: 0', axis =1)
all_plan.columns = [x.replace("all_", "") for x in all_plan.columns.values]
all_plan

,좌표,100_count,110_count,120_count,200_count,210_count,220_count,221_count,250_count,260_count,290_count,310_count,320_count,330_count,390_count,500_count,520_count,기타 사회복지시설_count,초등학교_count,유치원_count,기타 체육시설_count,중학교_count,기타 도서관시설_count,청사(국가)_count,고등학교_count,공공체육시설_count,특수학교_count,각종학교_count,기타 공공직업훈련시설_count,대학_count,기타 운동장시설_count,지방자치단체_count,골프장_count,기타 연구시설_count,기타청소년수련시설_count,기타 문화시설_count,공관_count,박물관_count,기타 학교시설_count,종합운동장_count,노외주차장_count,공영차고지_count,기타 주차장시설_count,일반철도_count,기타철도시설_count,도시철도_count,여객자동차터미널_count
0,"(37.57961579548528, 126.87506406643347)",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,"(37.57963617229605, 126.87619583846381)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,"(37.57965653826528, 126.87732761163377)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,"(37.58049614012486, 126.87390667102693)",0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,"(37.5805165284361, 126.8750384555327)",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20833,"(37.747040166676285, 126.91454518217792)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20834,"(37.74706024022562, 126.9156795278339)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20835,"(37.60951982027619, 126.83456514393055)",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


### 도시계획_Real(실재)

In [ ]:
real_plan

,Unnamed: 0,좌표,100_real_count,110_real_count,120_real_count,200_real_count,210_real_count,220_real_count,221_real_count,250_real_count,260_real_count,290_real_count,310_real_count,320_real_count,330_real_count,390_real_count,500_real_count,520_real_count,기타 사회복지시설_real_count,초등학교_real_count,유치원_real_count,기타 체육시설_real_count,중학교_real_count,기타 도서관시설_real_count,청사(국가)_real_count,고등학교_real_count,공공체육시설_real_count,특수학교_real_count,각종학교_real_count,기타 공공직업훈련시설_real_count,대학_real_count,기타 운동장시설_real_count,지방자치단체_real_count,골프장_real_count,기타 연구시설_real_count,기타청소년수련시설_real_count,기타 문화시설_real_count,공관_real_count,박물관_real_count,기타 학교시설_real_count,종합운동장_real_count,노외주차장_real_count,공영차고지_real_count,기타 주차장시설_real_count,일반철도_real_count,기타철도시설_real_count,도시철도_real_count,여객자동차터미널_real_count
0,0,"(37.57961579548528, 126.87506406643347)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,"(37.57963617229605, 126.87619583846381)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,"(37.57965653826528, 126.87732761163377)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,"(37.58049614012486, 126.87390667102693)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,"(37.5805165284361, 126.8750384555327)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20832,20832,"(37.74702008221923, 126.91341083764837)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20833,20833,"(37.747040166676285, 126.91454518217792)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20834,20834,"(37.74706024022562, 126.9156795278339)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20835,20835,"(37.60951982027619, 126.83456514393055)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
# 학습 시 동일할 칼럼이 필요하므로 칼럼명 동일하게 맞춰주기 
real_plan = real_plan.drop('Unnamed: 0', axis =1)
real_plan.columns = [x.replace("real_", "") for x in real_plan.columns.values]
real_plan

,좌표,100_count,110_count,120_count,200_count,210_count,220_count,221_count,250_count,260_count,290_count,310_count,320_count,330_count,390_count,500_count,520_count,기타 사회복지시설_count,초등학교_count,유치원_count,기타 체육시설_count,중학교_count,기타 도서관시설_count,청사(국가)_count,고등학교_count,공공체육시설_count,특수학교_count,각종학교_count,기타 공공직업훈련시설_count,대학_count,기타 운동장시설_count,지방자치단체_count,골프장_count,기타 연구시설_count,기타청소년수련시설_count,기타 문화시설_count,공관_count,박물관_count,기타 학교시설_count,종합운동장_count,노외주차장_count,공영차고지_count,기타 주차장시설_count,일반철도_count,기타철도시설_count,도시철도_count,여객자동차터미널_count
0,"(37.57961579548528, 126.87506406643347)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,"(37.57963617229605, 126.87619583846381)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,"(37.57965653826528, 126.87732761163377)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,"(37.58049614012486, 126.87390667102693)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,"(37.5805165284361, 126.8750384555327)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20833,"(37.747040166676285, 126.91454518217792)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20834,"(37.74706024022562, 126.9156795278339)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20835,"(37.60951982027619, 126.83456514393055)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
# 기존 lscp 
lscp_prev = pd.read_pickle('./모델학습용 데이터/Allpoint_lscp_all_300_도로명추가.pickle')
lscp_prev

,좌표,위도,경도,주거인구,버스_승하차수,지하철_승하차수,유동인구,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,CD_04402_사무소,CD_04403_부동산중개업소,CD_08102_중학교,...,CD_03014_치과의원,CD_12999_기타위락시설,CD_05301_예식장,CD_13200_공해공장,CD_19006_전신전화국,CD_04107_골프연습장,CD_07101_종합병원,CD_06204_대형점,CD_04022_멀티미디어문화콘텐츠 설비제공업소,CD_07102_산부인과병원,CD_06303_여객자동차터미널,CD_15006_액화가스판매소,CD_08601_청소년수련원(관),CD_15999_기타위험물저장처리시설,CD_04405_출판사,CD_16007_운전학원,CD_03019_탁구장,CD_03017_접골원,CD_03018_조산원,CD_04023_복합유통제공업소,CD_11299_기타관광숙박시설,CD_04008_삭진관,CD_09005_테니스장,CD_03106_전신전화국,CD_05303_공회당,CD_05699_기타동.식물원,CD_04399_기타공연장,CD_05403_체육관,CD_15007_유독물보관저장시설,CD_05505_산업전시장,CD_04021_세탁소,CD_09011_골프연습장,CD_20002_납골당,CD_06201_시장,CD_17105_부화장,CD_08699_기타생활권수련시설,CD_03109_공공도서관,CD_27999,CD_90002,CD_05502_미술관
0,"(37.57961579548528, 126.87506406643347)",37.579616,126.875064,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"(37.57963617229605, 126.87619583846381)",37.579636,126.876196,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"(37.57965653826528, 126.87732761163377)",37.579657,126.877328,0.0,0.0,0.0,0.0,21.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(37.58049614012486, 126.87390667102693)",37.580496,126.873907,0.0,0.0,0.0,0.0,17.0,1,0,0,0,0,0,0,0,0,3,0,9,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(37.5805165284361, 126.8750384555327)",37.580517,126.875038,0.0,0.0,0.0,0.0,16.0,1,0,0,0,0,0,0,0,0,3,0,7,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"(37.58053690590551, 126.87617024117878)",37.580537,126.876170,0.0,0.0,0.0,0.0,14.0,1,0,0,0,0,0,0,0,0,3,0,6,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"(37.58055727253304, 126.87730202796448)",37.580557,126.877302,0.0,0.0,0.0,0.0,13.0,1,0,0,0,0,0,0,0,0,3,0,4,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"(37.58106917422839, 126.85577243099591)",37.581069,126.855772,0.0,0.0,0.0,0.0,13.0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"(37.58108973666921, 126.85690421079029)",37.581090,126.856904,0.0,0.0,0.0,0.0,16.0,1,0,0,1,0,0,0,0,0,1,0,4,0,0,2,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"(37.581110288268384, 126.85803599173475)",37.581110,126.858036,0.0,0.0,0.0,0.0,17.0,1,0,0,1,0,0,0,0,0,1,0,3,0,0,2,2,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
lscp_new = lscp_prev.merge(all_plan, on = '좌표')
lscp_new 

,좌표,위도,경도,주거인구,버스_승하차수,지하철_승하차수,유동인구,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,CD_04402_사무소,CD_04403_부동산중개업소,CD_08102_중학교,...,221_count,250_count,260_count,290_count,310_count,320_count,330_count,390_count,500_count,520_count,기타 사회복지시설_count,초등학교_count,유치원_count,기타 체육시설_count,중학교_count,기타 도서관시설_count,청사(국가)_count,고등학교_count,공공체육시설_count,특수학교_count,각종학교_count,기타 공공직업훈련시설_count,대학_count,기타 운동장시설_count,지방자치단체_count,골프장_count,기타 연구시설_count,기타청소년수련시설_count,기타 문화시설_count,공관_count,박물관_count,기타 학교시설_count,종합운동장_count,노외주차장_count,공영차고지_count,기타 주차장시설_count,일반철도_count,기타철도시설_count,도시철도_count,여객자동차터미널_count
0,"(37.57961579548528, 126.87506406643347)",37.579616,126.875064,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,"(37.57963617229605, 126.87619583846381)",37.579636,126.876196,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,"(37.57965653826528, 126.87732761163377)",37.579657,126.877328,0.0,0.0,0.0,0.0,21.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,"(37.58049614012486, 126.87390667102693)",37.580496,126.873907,0.0,0.0,0.0,0.0,17.0,1,0,0,0,0,0,0,0,0,3,0,9,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,"(37.5805165284361, 126.8750384555327)",37.580517,126.875038,0.0,0.0,0.0,0.0,16.0,1,0,0,0,0,0,0,0,0,3,0,7,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,"(37.58053690590551, 126.87617024117878)",37.580537,126.876170,0.0,0.0,0.0,0.0,14.0,1,0,0,0,0,0,0,0,0,3,0,6,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,"(37.58055727253304, 126.87730202796448)",37.580557,126.877302,0.0,0.0,0.0,0.0,13.0,1,0,0,0,0,0,0,0,0,3,0,4,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,"(37.58106917422839, 126.85577243099591)",37.581069,126.855772,0.0,0.0,0.0,0.0,13.0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"(37.58108973666921, 126.85690421079029)",37.581090,126.856904,0.0,0.0,0.0,0.0,16.0,1,0,0,1,0,0,0,0,0,1,0,4,0,0,2,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,3,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"(37.581110288268384, 126.85803599173475)",37.581110,126.858036,0.0,0.0,0.0,0.0,17.0,1,0,0,1,0,0,0,0,0,1,0,3,0,0,2,2,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0,...,0,0,1,0,3,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
lscp_new.to_csv('./모델학습용 데이터/Allpoint_LSCP_ALL.csv')

-------------------
real plan

In [ ]:
real_plan

,좌표,100_count,110_count,120_count,200_count,210_count,220_count,221_count,250_count,260_count,290_count,310_count,320_count,330_count,390_count,500_count,520_count,기타 사회복지시설_count,초등학교_count,유치원_count,기타 체육시설_count,중학교_count,기타 도서관시설_count,청사(국가)_count,고등학교_count,공공체육시설_count,특수학교_count,각종학교_count,기타 공공직업훈련시설_count,대학_count,기타 운동장시설_count,지방자치단체_count,골프장_count,기타 연구시설_count,기타청소년수련시설_count,기타 문화시설_count,공관_count,박물관_count,기타 학교시설_count,종합운동장_count,노외주차장_count,공영차고지_count,기타 주차장시설_count,일반철도_count,기타철도시설_count,도시철도_count,여객자동차터미널_count
0,"(37.57961579548528, 126.87506406643347)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,"(37.57963617229605, 126.87619583846381)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,"(37.57965653826528, 126.87732761163377)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,"(37.58049614012486, 126.87390667102693)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,"(37.5805165284361, 126.8750384555327)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20832,"(37.74702008221923, 126.91341083764837)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20833,"(37.747040166676285, 126.91454518217792)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20834,"(37.74706024022562, 126.9156795278339)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20835,"(37.60951982027619, 126.83456514393055)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
# 기존 lscp 
lscp_prev = pd.read_pickle('./모델학습용 데이터/Allpoint_lscp_all_300_도로명추가.pickle')
lscp_prev

,좌표,위도,경도,주거인구,버스_승하차수,지하철_승하차수,유동인구,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,CD_04402_사무소,CD_04403_부동산중개업소,CD_08102_중학교,...,CD_03014_치과의원,CD_12999_기타위락시설,CD_05301_예식장,CD_13200_공해공장,CD_19006_전신전화국,CD_04107_골프연습장,CD_07101_종합병원,CD_06204_대형점,CD_04022_멀티미디어문화콘텐츠 설비제공업소,CD_07102_산부인과병원,CD_06303_여객자동차터미널,CD_15006_액화가스판매소,CD_08601_청소년수련원(관),CD_15999_기타위험물저장처리시설,CD_04405_출판사,CD_16007_운전학원,CD_03019_탁구장,CD_03017_접골원,CD_03018_조산원,CD_04023_복합유통제공업소,CD_11299_기타관광숙박시설,CD_04008_삭진관,CD_09005_테니스장,CD_03106_전신전화국,CD_05303_공회당,CD_05699_기타동.식물원,CD_04399_기타공연장,CD_05403_체육관,CD_15007_유독물보관저장시설,CD_05505_산업전시장,CD_04021_세탁소,CD_09011_골프연습장,CD_20002_납골당,CD_06201_시장,CD_17105_부화장,CD_08699_기타생활권수련시설,CD_03109_공공도서관,CD_27999,CD_90002,CD_05502_미술관
0,"(37.57961579548528, 126.87506406643347)",37.579616,126.875064,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"(37.57963617229605, 126.87619583846381)",37.579636,126.876196,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"(37.57965653826528, 126.87732761163377)",37.579657,126.877328,0.0,0.0,0.0,0.0,21.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(37.58049614012486, 126.87390667102693)",37.580496,126.873907,0.0,0.0,0.0,0.0,17.0,1,0,0,0,0,0,0,0,0,3,0,9,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(37.5805165284361, 126.8750384555327)",37.580517,126.875038,0.0,0.0,0.0,0.0,16.0,1,0,0,0,0,0,0,0,0,3,0,7,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"(37.58053690590551, 126.87617024117878)",37.580537,126.876170,0.0,0.0,0.0,0.0,14.0,1,0,0,0,0,0,0,0,0,3,0,6,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"(37.58055727253304, 126.87730202796448)",37.580557,126.877302,0.0,0.0,0.0,0.0,13.0,1,0,0,0,0,0,0,0,0,3,0,4,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"(37.58106917422839, 126.85577243099591)",37.581069,126.855772,0.0,0.0,0.0,0.0,13.0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"(37.58108973666921, 126.85690421079029)",37.581090,126.856904,0.0,0.0,0.0,0.0,16.0,1,0,0,1,0,0,0,0,0,1,0,4,0,0,2,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"(37.581110288268384, 126.85803599173475)",37.581110,126.858036,0.0,0.0,0.0,0.0,17.0,1,0,0,1,0,0,0,0,0,1,0,3,0,0,2,2,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
lscp_new = lscp_prev.merge(real_plan, on = '좌표')
lscp_new

,좌표,위도,경도,주거인구,버스_승하차수,지하철_승하차수,유동인구,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,CD_04402_사무소,CD_04403_부동산중개업소,CD_08102_중학교,...,221_count,250_count,260_count,290_count,310_count,320_count,330_count,390_count,500_count,520_count,기타 사회복지시설_count,초등학교_count,유치원_count,기타 체육시설_count,중학교_count,기타 도서관시설_count,청사(국가)_count,고등학교_count,공공체육시설_count,특수학교_count,각종학교_count,기타 공공직업훈련시설_count,대학_count,기타 운동장시설_count,지방자치단체_count,골프장_count,기타 연구시설_count,기타청소년수련시설_count,기타 문화시설_count,공관_count,박물관_count,기타 학교시설_count,종합운동장_count,노외주차장_count,공영차고지_count,기타 주차장시설_count,일반철도_count,기타철도시설_count,도시철도_count,여객자동차터미널_count
0,"(37.57961579548528, 126.87506406643347)",37.579616,126.875064,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,"(37.57963617229605, 126.87619583846381)",37.579636,126.876196,0.0,0.0,0.0,0.0,25.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,"(37.57965653826528, 126.87732761163377)",37.579657,126.877328,0.0,0.0,0.0,0.0,21.0,1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,"(37.58049614012486, 126.87390667102693)",37.580496,126.873907,0.0,0.0,0.0,0.0,17.0,1,0,0,0,0,0,0,0,0,3,0,9,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,"(37.5805165284361, 126.8750384555327)",37.580517,126.875038,0.0,0.0,0.0,0.0,16.0,1,0,0,0,0,0,0,0,0,3,0,7,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,"(37.58053690590551, 126.87617024117878)",37.580537,126.876170,0.0,0.0,0.0,0.0,14.0,1,0,0,0,0,0,0,0,0,3,0,6,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,"(37.58055727253304, 126.87730202796448)",37.580557,126.877302,0.0,0.0,0.0,0.0,13.0,1,0,0,0,0,0,0,0,0,3,0,4,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,"(37.58106917422839, 126.85577243099591)",37.581069,126.855772,0.0,0.0,0.0,0.0,13.0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"(37.58108973666921, 126.85690421079029)",37.581090,126.856904,0.0,0.0,0.0,0.0,16.0,1,0,0,1,0,0,0,0,0,1,0,4,0,0,2,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"(37.581110288268384, 126.85803599173475)",37.581110,126.858036,0.0,0.0,0.0,0.0,17.0,1,0,0,1,0,0,0,0,0,1,0,3,0,0,2,2,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
lscp_new.to_csv('./모델학습용 데이터/Allpoint_LSCP_REAL.csv', index = False)